In [6]:
!python3 labelstudio2doccano.py --labelstudio_file test_label_studio.json

In [12]:
import gc
gc.collect()

30816

In [3]:
!python3 doccano.py \
    --doccano_file doccano_ext.jsonl \
    --task_type ext \
    --save_dir .\
    --seed 20230418 \
    --splits 0.7 0.2 0.1 \
    --schema_lang ch \
    --negative_ratio 5 #負樣本比例

[2023-04-18 08:14:56,344] [    INFO] - Converting doccano data...
100%|██████████████████████████████████████████| 5/5 [00:00<00:00, 22623.00it/s]
[2023-04-18 08:14:56,345] [    INFO] - Adding negative samples for first stage prompt...
100%|█████████████████████████████████████████| 5/5 [00:00<00:00, 117159.33it/s]
[2023-04-18 08:14:56,345] [    INFO] - Converting doccano data...
100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 24174.66it/s]
[2023-04-18 08:14:56,346] [    INFO] - Adding negative samples for first stage prompt...
100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 66052.03it/s]
[2023-04-18 08:14:56,346] [    INFO] - Converting doccano data...
100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 16320.25it/s]
[2023-04-18 08:14:56,346] [    INFO] - Adding negative samples for first stage prompt...
100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 39568.91it/s]
[2023-04-18 08:14:56,348] [    INFO] - Save 15 ex

In [19]:
import json, os
import numpy as np
fileList = ["train.txt", "test.txt", "dev.txt"]
origin = ["求償慰撫金", "精神慰撫金", "其他慰撫金"]
v1 = ["求償精神慰撫金", "裁定精神慰撫金", "其他精神慰撫金"]
v2 = ["請求精神慰撫金", "裁定精神慰撫金", "其他精神慰撫金"]
#感覺用完全相同的字，模型也不容易區分出慰撫金的類別，以下試試看求償和裁定用不同的敘述
#因為請求和裁定其實大約落在不同段落，因此透過該段落出現過的字詞來描述：
v3 = ["非財產上之損害的求償精神慰撫金", "法院得心證判斷之適當裁定精神慰撫金", "其他精神慰撫金"]
def change_prompt(
    origin_type: list,  
    change_type: list,
    change_file: str,
    out_file: str
    ):
    
    out = []
    with open(change_file, "r") as f:
        lines = f.readlines()
        for line in lines:
            
            f = json.loads(line)
            
            for i, s in enumerate(origin_type):
                if f["prompt"].find(s) > -1:
                    f["prompt"] = f["prompt"].replace(s, change_type[i])
                    break
            
            out.append(json.dumps(f, ensure_ascii=False))
            
            #print(out)
            
        dir = os.getcwd() + out_file
        directory = os.path.dirname(dir)
        if not os.path.exists(directory):
            os.makedirs(directory)
        np.savetxt(dir, out, fmt='%s')
        #print(out[0])

In [ ]:
#新增v1 ~ v3
for i in ["v1", "v2", "v3"]:
    for file in fileList:
        change_prompt(origin_type=origin, 
                change_type=eval(i),
                change_file=file,
                out_file="/data_" + i + "/" + file)

In [20]:
#新增v4
v4 = ["原告主張非財產上之損害的求償精神慰撫金", "法院得心證判斷之適當裁定精神慰撫金", "其他精神慰撫金"]
for file in fileList:
    change_prompt(origin_type=origin, 
            change_type=v4,
            change_file=file,
            out_file="/data_v4/" + file)

In [ ]:
!python3 finetune.py  \
    --device gpu \
    --logging_steps 10 \
    --save_steps 100 \
    --eval_steps 100 \
    --seed 87 \
    --model_name_or_path uie-base  \
    --output_dir ./data_v1/checkpoint/model_best \
    --train_path ./data_v1/train.txt \
    --dev_path ./data_v1/dev.txt  \
    --max_seq_length 512  \
    --per_device_eval_batch_size 16\
    --per_device_train_batch_size  16 \
    --num_train_epochs 3 \
    --learning_rate 1e-5 \
    --label_names 'start_positions' 'end_positions' \
    --do_train \
    --do_eval \
    --do_export \
    --export_model_dir ./data_v1/checkpoint/model_best \
    --overwrite_output_dir \
    --disable_tqdm True \
    --metric_for_best_model eval_f1 \
    --load_best_model_at_end  True \
    --save_total_limit 1

In [ ]:
#case v2
!export finetuned_model=./data_v2/checkpoint/model_best

!python3 finetune.py  \
    --device gpu \
    --logging_steps 10 \
    --save_steps 100 \
    --eval_steps 100 \
    --seed 87 \
    --model_name_or_path uie-base  \
    --output_dir ./data_v2/checkpoint/model_best \
    --train_path ./data_v2/train.txt \
    --dev_path ./data_v2/dev.txt  \
    --max_seq_length 512  \
    --per_device_eval_batch_size 16\
    --per_device_train_batch_size  16 \
    --num_train_epochs 3 \
    --learning_rate 1e-5 \
    --label_names 'start_positions' 'end_positions' \
    --do_train \
    --do_eval \
    --do_export \
    --export_model_dir ./data_v2/checkpoint/model_best \
    --overwrite_output_dir \
    --disable_tqdm True \
    --metric_for_best_model eval_f1 \
    --load_best_model_at_end  True \
    --save_total_limit 1

In [ ]:
#case v3
!export finetuned_model=./data_v3/checkpoint/model_best

!python3 finetune.py  \
    --device gpu \
    --logging_steps 10 \
    --save_steps 100 \
    --eval_steps 100 \
    --seed 87 \
    --model_name_or_path uie-base  \
    --output_dir ./data_v3/checkpoint/model_best \
    --train_path ./data_v3/train.txt \
    --dev_path ./data_v3/dev.txt  \
    --max_seq_length 512  \
    --per_device_eval_batch_size 16\
    --per_device_train_batch_size  16 \
    --num_train_epochs 3 \
    --learning_rate 1e-5 \
    --label_names 'start_positions' 'end_positions' \
    --do_train \
    --do_eval \
    --do_export \
    --export_model_dir ./data_v3/checkpoint/model_best \
    --overwrite_output_dir \
    --disable_tqdm True \
    --metric_for_best_model eval_f1 \
    --load_best_model_at_end  True \
    --save_total_limit 1

In [ ]:
#case v4
!export finetuned_model=./data_v4/checkpoint/model_best

!python3 finetune.py  \
    --device gpu \
    --logging_steps 10 \
    --save_steps 100 \
    --eval_steps 100 \
    --seed 87 \
    --model_name_or_path uie-base  \
    --output_dir ./data_v4/checkpoint/model_best \
    --train_path ./data_v4/train.txt \
    --dev_path ./data_v4/dev.txt  \
    --max_seq_length 512  \
    --per_device_eval_batch_size 16\
    --per_device_train_batch_size  16 \
    --num_train_epochs 3 \
    --learning_rate 1e-5 \
    --label_names 'start_positions' 'end_positions' \
    --do_train \
    --do_eval \
    --do_export \
    --export_model_dir ./data_v4/checkpoint/model_best \
    --overwrite_output_dir \
    --disable_tqdm True \
    --metric_for_best_model eval_f1 \
    --load_best_model_at_end  True \
    --save_total_limit 1

In [4]:
#origin
!export finetuned_model=./checkpoint/model_best

!python3 finetune.py  \
    --device gpu \
    --logging_steps 10 \
    --save_steps 100 \
    --eval_steps 100 \
    --seed 42 \
    --model_name_or_path uie-base  \
    --output_dir ./checkpoint/model_best \
    --train_path train.txt \
    --dev_path dev.txt  \
    --max_seq_length 512  \
    --per_device_eval_batch_size 16\
    --per_device_train_batch_size  16 \
    --num_train_epochs 10 \
    --learning_rate 1e-5 \
    --label_names 'start_positions' 'end_positions' \
    --do_train \
    --do_eval \
    --do_export \
    --export_model_dir ./checkpoint/model_best \
    --overwrite_output_dir \
    --disable_tqdm True \
    --metric_for_best_model eval_f1 \
    --load_best_model_at_end  True \
    --save_total_limit 1


[2023-04-18 08:15:16,609] [ WARNING] - evaluation_strategy reset to IntervalStrategy.STEPS for do_eval is True. you can also set evaluation_strategy='epoch'.
[2023-04-18 08:15:16,609] [    INFO] - The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
[2023-04-18 08:15:16,609] [    INFO] - ============================================================
[2023-04-18 08:15:16,609] [    INFO] -      Model Configuration Arguments      
[2023-04-18 08:15:16,609] [    INFO] - paddle commit id              :4596b9a22540fb0ea5d369c3c804544de61d03d0
[2023-04-18 08:15:16,610] [    INFO] - export_model_dir              :./checkpoint/model_best
[2023-04-18 08:15:16,610] [    INFO] - model_name_or_path            :uie-base
[2023-04-18 08:15:16,610] [    INFO] - multilingual                  :F

## Evaluate

In [5]:
#origin
!python3 evaluate.py \
    --model_path ./checkpoint/model_best \
    --test_path test.txt \
    --batch_size 16 \
    --max_seq_len 512

#v1
'''
!python3 evaluate.py \
    --model_path ./data_v1/checkpoint/model_best \
    --test_path data_v1/test.txt \
    --batch_size 16 \
    --max_seq_len 512
'''

#v2
'''
!python3 evaluate.py \
    --model_path ./data_v2/checkpoint/model_best \
    --test_path data_v2/test.txt \
    --batch_size 16 \
    --max_seq_len 512
'''
'''
#v3
!python3 evaluate.py \
    --model_path ./data_v3/checkpoint/model_best \
    --test_path data_v3/test.txt \
    --batch_size 16 \
    --max_seq_len 512
'''


[2023-04-18 08:18:50,493] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load './checkpoint/model_best'.
[2023-04-18 08:18:50,515] [    INFO] - loading configuration file ./checkpoint/model_best/config.json
[2023-04-18 08:18:50,516] [    INFO] - Model config ErnieConfig {
  "architectures": [
    "UIE"
  ],
  "attention_probs_dropout_prob": 0.1,
  "dtype": "float32",
  "enable_recompute": false,
  "fuse": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "ernie",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "paddlenlp_version": null,
  "pool_act": "tanh",
  "task_id": 0,
  "task_type_vocab_size": 3,
  "type_vocab_size": 4,
  "use_task_id": true,
  "vocab_size": 40000
}

W0418 08:18:51.774190  4673 gpu_resources.cc:61] Please NOTE: device: 0,

'\n#v3\n!python3 evaluate.py     --model_path ./data_v3/checkpoint/model_best     --test_path data_v3/test.txt     --batch_size 16     --max_seq_len 512\n'

## Inference

In [11]:
#schema = ["原告", "精神慰撫金", '求償慰撫金', {'原告': '精神慰撫金'}, {'原告': '求償慰撫金'}]
#v1 幾乎多人的也能掌握
#schema = ["原告", "裁定精神慰撫金", '求償精神慰撫金', {'原告': '裁定精神慰撫金'}, {'原告': '求償精神慰撫金'}]
#v2
#schema = ["原告", "裁定精神慰撫金", '請求精神慰撫金', {'原告': '裁定精神慰撫金'}, {'原告': '請求精神慰撫金'}]
#v3
#schema = ["原告", "法院得心證判斷之適當裁定精神慰撫金", '非財產上之損害的求償精神慰撫金', "其他精神慰撫金", {'原告': '法院得心證判斷之適當裁定精神慰撫金'}, {'原告': '非財產上之損害的求償精神慰撫金'}]
#v4
#schema = ["原告", "法院得心證判斷之適當裁定精神慰撫金", '原告主張非財產上之損害的求償精神慰撫金', {'原告': '法院得心證判斷之適當裁定精神慰撫金'}, {'原告': '原告主張非財產上之損害的求償精神慰撫金'}]
schema = ["薪資費用", "醫療費用", "精神慰撫金額"]

mytext = "臺灣橋頭地方法院民事判決108年度訴字第851號原告劉麗香訴訟代理人陳煜昇律師被告陳志豪訴訟代理人吳燕龍上列被告因過失傷害案件，經原告提起刑事附帶民事訴訟請求損害賠償（本院108年度審交附民字第206號），經本院刑事庭裁定移送前來，本院於民國109年5月20日言詞辯論終結，判決如下：主文被告應給付原告新臺幣貳拾柒萬捌仟陸佰零柒元，及自民國一○八年六月二十七日起至清償日止按週年利率百分之五計算之利息。原告其餘之訴駁回。訴訟費用由原告負擔百分之七十二，餘由被告負擔。本判決原告勝訴部分得假執行，但被告如以新臺幣貳拾柒萬捌仟陸佰零柒元為原告預供擔保後，得免為假執行。原告其餘假執行之聲請駁回。事實及理由一、原告主張：被告甲○○於民國107年4月26日14時許，駕駛車牌號碼000-0000號自用小客貨車，沿高雄市岡山區介壽東路快車道由東往西方向行駛，行經介壽東路與岡燕路口時，本應注意依號誌行駛，竟疏未注意而違規右轉，未讓騎乘車牌號碼000-0000號普通重型機車（下稱系爭機車）而屬直行車之原告先行，兩車因而發生擦撞，致原告人車倒地，受有胸壁挫傷、頭部損傷、大腿、小腿及踝之表淺損傷、前臂挫傷、手挫傷等傷害，並導致原告膝關節病變，因此需進行手術、長期至骨科、身心科就診及需停業1年，精神上痛苦不堪，且因而受有醫療費用新臺幣（下同）215,774元、交通費用11,800元、自107年4月27日起至同年5月3日止及自107年7月12日起至同年月14日止之看護費用32,000元、1年短少之工作損失331,200元、系爭機車修理費用21,000元、眼鏡7,000元、安全帽1,200元、手錶維修費用700元等損害，另受有非財產上損害即精神慰撫金658,216元，扣除已領強制險保險金86,256元，應得請求被告賠償1,192,634元。爰依民法第184條、第193條、第195條、第196條、第213條、民法第229條第2項、第233條第1項前段、第203條規定請求被告賠償等語，並聲明：㈠被告應賠償原告1,192,634元，並自起訴狀繕本送達之翌日起至清償日止，按週年利率5％計算之利息；㈡請准供擔保宣告假執行。二、被告則以：原告膝關節之傷勢與本件事故無關，是除就原告於岡山醫院就診支出醫療費用1,150元不爭執外，其餘原告因膝關節傷勢至義大醫院就診之醫療費用、交通費用均與系爭事故無涉，且原告因系爭事故所受傷害僅為挫傷、擦傷，自無停業損失或需專人看護情事，另原告請求精神慰撫金金額亦過高。至系爭機車修理費用應予折舊計算，其餘眼鏡、安全帽、手錶毀損部分，未見原告舉證有何損壞情事及與系爭事故有何關聯，自不應請求被告賠償。又原告已申請強制險理賠，應扣除理賠金額等語資為抗辯，並聲明：㈠原告之訴及其假執行之聲請均駁回；㈡如受不利判決，願供擔保免予假執行。三、本件經法官會同兩造整理爭點，結果如下：㈠不爭執事項（本院採為判決之基礎）：⒈被告於107年4月26日14時許，駕駛車牌號碼000-0000號自用小客貨車，沿高雄市岡山區介壽東路快車道由東往西方向行駛，行經介壽東路與岡燕路口時，本應注意駕駛人駕駛汽車，應注意道路交通號誌之指示，而依當時天候晴、日間自然光線、路面乾燥無缺陷亦無障礙物，且視距良好，客觀上並無令其不能注意之特別情事，竟疏未注意該路段為多時相號誌，且快車道號誌右轉燈號已熄滅而變換為直行燈號，不得再右轉，即貿然右轉欲駛入岡燕路，適有原告騎乘系爭機車沿介壽東路由東往西方向慢車道駛至該處，兩車因而發生擦撞，原告因而人車倒地，致原告受有胸壁挫傷、頭部損傷、大腿、小腿及踝之表淺損傷、前臂挫傷、手挫傷等傷害。而原告就本件事故之發生則無過失。⒉原告於本件交通事故發生後有下列支出：⑴醫療費用：①於岡山醫院支出醫療費用1,150元，且屬因本件事故支出之必要費用。②於義大醫院支出醫療費用214,624元，但是否與本件事故有關被告有爭執。惟如與本件事故有關時，該部分費用亦屬必要費用，兩造均不爭執。⑵看護費用：原告有於107年4月27日至同年5月3日及107年7月12日至同年月19日聘請專人看護而分別支出看護費用14,000元及18,000元，但必要性及與本件事故是否有間被告有爭執。⑶機車修理費用21,000元，且均係因本件事故而受損。⑷手錶維修費用700元，但與本件事故是否有關被告有爭執。⒊原告因本件事故已領取強制汽車責任保險理賠86,256元。⒋原告曾經義大醫院診斷患有膝關節病變，並於107年7月12日入住，於同年月13日接受關節鏡清創手術，因病情需要須使用關節鏡耗材和自體高濃度血小板注射，於同年月14日出院，經醫師醫囑需休養復建至少半年。⒌系爭機車為原告所有，且為105年8月出廠。㈡爭執事項：⒈原告請求因膝關節病變支出之醫療費、看護費、交通費及工作損失有無理由？⑴原告前開罹患之膝關節病變與本件事故有無因果關係？⑵原告於107年7月12日至同年月19日有無聘請專人看護之必要？⑶原告是否因前往義大醫院就診而支出交通費用11,800元？該部分費用是否屬因本件事故增加之必要支出？⑷原告是否因前開膝關節病變而受有不能工作損失331,200元？①原告有無因前開膝關節病變而不能工作之情形？②原告如有，其期間為何？原告之薪資應如何計算？⒉原告因本件事故所受如不爭執事項⒈之傷勢，有無於107年4月27日至同年5月3日聘請專人看護照顧之必要？⒊原告所有之車牌號碼000-0000號普通重型機車修復費用是否應予折舊？得請求之必要修復費用為何？⒋原告是否因本件事故受有眼鏡7,000元、安全帽1,200元及手錶維修費用700元之損害？如有，損害金額是否應折舊計算？⒌原告得請求之慰撫金數額為何？⒍綜上，經扣除強制汽車責任保險理賠後，原告得請求之賠償數額為何？四、得心證之理由：㈠按汽車、機車或其他非依軌道行駛之動力車輛，在使用中加損害於他人者，駕駛人應賠償因此所受之損害，民法第191條之2前段定有明文。次按駕駛人駕汽車，應注意道路交通號誌之指示，則為道路交通安全規則第90條第1項所明定。本件兩造間於不爭執事項⒈所載時、地，因被告未依號誌行駛即貿然右轉而與原告發生交通事故等情，為兩造所不爭執，而被告既為領有普通小型車駕駛執照之人，自應注意上開規定，亦為其應注意之義務。而依當時當時天候晴、日間自然光線、路面乾燥無缺陷亦無障礙物，且視距良好，客觀上並無令其不能注意之特別情事，被告竟未遵守號誌而右轉致發生本件事故，並因而導致原告因而倒地受傷，被告就本件事故自有過失，揆諸上開規定，原告應得請求被告賠償因而所受之損害。㈡又按不法侵害他人之身體或健康者，對於被害人因此喪失或減少勞動能力或增加生活上之需要時，應負損害賠償責任，民法第193條第1項定有明文。經查：⒈醫療費用部分：⑴岡山醫院部分：原告因本件事故受有岡山醫院支出醫療費用1,150元，且屬因本件事故支出之必要費用，為兩造所不爭執，原告自得請求被告賠償該部分損害。⑵義大醫院部分：原告主張有因左膝關節病變之傷害，於義大醫院支出醫療費用214,624元等情，雖為被告所不爭執，惟原告主張該傷勢係因本件事故所致部分，則為被告所否認，經查：①原告於本件事故發生之107年4月26日當日至岡山醫院急診就診時，確有主訴其左膝疼痛之情，有岡山醫院病歷在卷可參（見本院卷第64頁），且其於相隔甚近之107年5月1日至義大醫院就診時，就其左膝部分之傷勢，亦有主訴係於107年4月26日之車禍事故受有頭部及左膝外傷，且經診斷為左膝新傷等節，業據義大醫院109年4月23日義大醫院字第10900704號函說明甚詳（見本院卷第123頁），本院審酌本件事故係於107年4月26日發生，與原告於107年5月1日至義大醫院就診時相隔甚近，足徵原告主張其左膝之傷勢係因本件事故所致，應屬有據。至被告雖辯稱原告於至義大醫院就診後並未為任何治療，相隔2個月後始再入院接受手術，應認該傷勢與本件事故無關云云，然原告於107年5月1日至義大醫院就診後，即接續於同年月22日、29日、同年7月10月至義大醫院就診，有義大醫院病歷資料在卷可參（見審交易卷第79至85頁），足見原告確有接續因該傷勢就診之情形，自難認原告就傷勢之處置有合異於常情之處，尚難以此推認該傷勢與本件事故無關，被告此部分辯解，並無理由。②又本院刑事庭雖曾函詢岡山醫院原告所受膝關節傷勢是否與本件事故有關，經該院函覆稱：原告107年4月26日就診時確有左膝紅腫疼痛之症狀，影像檢查顯示無骨折，但無從確認軟組織（韌帶、半月板等）有無損傷，上開傷勢確有可能與107年4月26日車禍事故有關，但無法確認此期間內告訴人是否有其他外傷，故只能說可能有關，但無法完全肯定等語（見審交易卷第65頁），是依上開岡山醫院回函，雖無法完全肯定原告所受左膝傷勢之成因，但確有可能係因本件事故導致，更堪認原告主張其左膝傷勢係因本件事故所致，應非無憑。③此外，經本院依原告之聲請函調原告於107年4月12日至22日間之健保就診記錄，亦僅有於107年4月13日至義大醫院就診之記錄，有衛生福利部中央健康保險署109年4月10日健保高字第1096129722號函暨所附就醫記錄在卷可參（見本院卷第117至121頁），且依義大醫院上開回函，原告並無於107年4月13日至該院骨科就診之記錄，顯見該次就診應非至骨科就診，應足以排除係因左膝傷勢而就醫，自亦難認原告於107年5月1日前2週內，另有因左膝傷勢而就診之情形，應無從排除原告左膝之傷勢係因本件事故所致。④而被告就此亦未提出其他反證足以推翻原告上開舉證，自應認原告左膝關節病變之傷勢，確係因本件事故造成。而該傷勢如與本件事故有關時，該部分費用亦屬必要費用，為兩造所不爭執，原告自得就其因左膝關節病變至義大醫院就診之醫療費用214,624元請求被告賠償。⑶綜上，原告得請求之醫療費用即為215,774元（計算式：1,150＋214,624＝215,774）。⒉看護費用部分：原告有於107年4月27日至同年5月3日及107年7月12日至同年月19日聘請專人看護而分別支出看護費用14,000元及18,000元等節，為兩造所不爭執，堪信屬實。而就有無專人看護必要部分，其中因左膝關節之傷勢，於107年7月12日起需受人全日看護1個月，有義大醫療財團法人義大癌治療醫院109年2月6日義大癌治療字第10900043號函在卷可參（見本院卷第73頁），足徵原告請求107年7月12日至同年月19日之看護費18,000元，應屬有據。原告至岡山醫院就診之其餘傷勢部分，則無專人看護之問題，業據岡山醫院108年12月26日岡秀醫字第0000363號函所附醫理見解說明甚詳（見本院卷第71頁），是原告請求107年4月27日至同年5月3日間之看護費用14,000元，應屬無據。⒊交通費部分：原告雖主張其因本件事故所受之傷勢，需乘坐計程車往返家中及醫院，因而支出交通費用11,800元，惟為被告所否認，且原告亦未能提出實際有增加此一支出之相關單據佐證，自難為有利原告之認定。至原告雖聲請本院函詢高雄市計程車客運商業同業公會自原告住處至義大醫院單程所需計程車交通費用為何，然原告既已未能證明有此一支出，亦無從僅以推估之費用認定原告確有增加此一生活上費用，原告主張受有此部分損害，仍屬不能證明，自亦無從請求被告賠償。㈢不能工作損失部分：⒈就原告有因本件事故所致傷勢不能工作之情形部分，原告雖主張不能工作期間為12個月云云，然經本院分別函詢岡山醫院及義大醫院，其中岡山醫院部分，醫師僅提及受傷當下應休息，並未提及後續有無應持續休養之必要，有岡山醫院前開回函暨所附醫理見解說明可參（見本院卷第69至71頁），亦僅能認至多僅有受傷當日有不能工作情形。而就義大醫院部分，則經醫師評估自107年7月13日手術後2個月內，因行動不便，須休養而無法工作，亦有義大癌治療醫院前開回函在卷可佐，堪認原告確有應休養2個月不能工作之情事，合計應休養而不能工作之期間即為2月又1日。⒉就原告薪資部分，原告雖主張其每月薪資應以27,600元計算，並提出勞工保險被保險人投保資料表為證（見附民卷第53頁），然原告係投保於屏東縣清潔服務職業工會，並非投保於公司行號，該投保薪資自難以回推原告實際之所得，原告復未提出其他證據證明其每月薪資確有27,600元，自難為有利原告之認定。惟本院審酌原告於事故當時僅為56歲，尚有工作之能力，應至少可獲得最低薪資之所得，故至少應得以當時之最低薪資22,000元計算原告每月之工作所得，故原告得請求之不能工作損失即為22,733元（計算式：22,000×（1＋1/30）＝22,733，小數點以下四捨五入）。㈣系爭機車修理費用部分：⒈按不法毀損他人之物者，被害人得請求賠償其物因毀損所減少之價額；負損害賠償責任者，除法律另有規定或契約另有訂定外，應回復他方損害發生前之原狀。因回復原狀而應給付金錢者，自損害發生時起，加給利息。第1項情形，債權人得請求支付回復原狀所必要之費用，以代回復原狀；不能回復原狀或回復顯有重大困難者，應以金錢賠償其損害，民法第196條、第213條、第215條分別定有明文。又物被毀損時，被害人除得依民法第196條請求賠償外，並不排除民法第213條至第215條之適用。依民法第196條請求賠償物被毀損所減少之價額，得以修復費用為估定之標準，但以必要者為限，例如：修理材料以新品換舊品，應予折舊（最高法院77年度第9次民事庭會議決議亦同此見解）。⒉本件原告主張其所有之系爭機車係於105年8月出廠，且因本件事故受損，因而支出修理費用21,000元等情，為兩造所不爭執，堪信屬實。又依原告提出之系爭機車修理單據（見附民卷第65頁）之記載，系爭機車之維修費用中，僅有車台校正2,000元及工資1,500元部分得認為屬於工資，其餘部分17,500元則均為零件費用，亦堪認定。而系爭機車既為105年8月出廠，算至本件車禍發生時之107年4月26日，使用期間為1年9月。而系爭機車之修理既有以新零件更換被損之舊零件，參照上開最高法院民事庭決議之意旨，自應將零件折舊部分予以扣除。依行政院所頒固定資產耐用年數表及固定資產折舊率之規定，機械腳踏車之耐用年數為3年，依定率遞減法每年折舊1000分之536，另依營利事業所得稅查核準則第95條第6項規定「固定資產提列折舊採用定率遞減法者，以1年為計算單位，其使用期間未滿1年者，按實際使用之月數相當於全年之比例計算之，不滿1月者，以1月計」，則零件扣除折舊後之修復費用估定為4,856元（計算式：第一年折舊值17,500×0.536＝9,380，折舊後價值17,500－9,380＝8,120，第二年折舊值8,120×0.536×9/12＝3,264，小數點以下四捨五入，折舊後價值8,120－3,264＝4,856），再加計毋庸折舊之工資3,500元，總計必要之修理費用應為8,356元（計算式：4,856＋3,500＝8,356），即為原告得請求賠償之金額。㈤眼鏡、安全帽及手錶維修費用部分：原告雖主張其所有之眼鏡、安全帽及手錶因本件事故而毀損等情，惟為被告所否認，原告復未能提出任何足以證明其上開物品確有因本件事故而受損之證據，自難為有利原告之認定，而僅能上開物品並無因本件事故受損，自無從請求被告賠償。㈥慰撫金部分：按不法侵害他人之身體、健康、名譽、自由、信用、隱私、貞操，或不法侵害其他人格法益而情節重大者，被害人雖非財產上之損害，亦得請求賠償相當之金額，其名譽被侵害者，並得請求回復名譽之適當處分，民法第195條第1項定有明文。又按慰藉金之賠償須以人格權遭遇侵害，使精神上受有痛苦為必要，其核給之標準固與財產上損害之計算不同，然非不可斟酌雙方身分資力與加害程度，及其他各種情形核定相當之數額（最高法院51年台上字第223號判例意旨參照）。亦即民法第195條不法侵害他人人格法益得請求賠償相當之金額，應以實際加害之情形、加害之程度、被害人所受精神上痛苦之程度、身分、地位、經濟能力，並斟酌賠償義務人之故意或過失之可歸責事由、資力或經濟狀況綜合判斷之。本件被告既有因過失侵害原告之身體權情事，原告自得請求被告給付非財產上損害。查本件原告為國小畢業，曾任工廠女工及清潔公司員工，每月收入約30,000元；被告則為科技大學畢業，曾任工地主任，從事營造業等情，業據兩造分別陳報在案。爰審酌本件原告所受傷勢程度，另審酌兩造之職業、教育程度、所得狀況等一切情狀，認原告請求被告給付658,216元之慰撫金，尚屬過高，應以100,000元為適當。從而，原告所受之損害數額，即為364,863元（計算式：215,774＋18,000＋22,733＋8,356＋100,000＝364,863）。㈦惟按保險人依本法規定所為之保險給付，視為被保險人損害賠償金額之一部分；被保險人受賠償請求時，得扣除之，為強制汽車責任保險法第32條所明定。本件原告由強制汽車責任保險獲理賠86,256元，為兩造所不爭執，依上開規定，自得就原告得請求之金額扣除，是以原告得請求賠償之數額扣除強制汽車責任保險理賠後應為278,607元（計算式：364,863－86,256＝278,607）。五、綜上所述，被告雖因過失而侵害原告之身體權，並造成原告所有之系爭機車受損，而應賠償原告之損失，惟原告得請求之數額經扣除強制汽車責任保險理賠後應為278,607元。是原告依民法第184條、第193條、第195條、第196條、第213條、民法第229條第2項、第233條第1項前段、第203條規定請求被告賠償，於請求被告給付278,607元，及自起訴狀繕本送達翌日即108年6月27日起至清償日止按週年利率5％計算利息之範圍內，為有理由，應予准許，逾此部分之請求，則無理由，應予駁回。六、本件原告勝訴部分，為所命給付金額未逾500,000元之判決，依民事訴訟法第389條第1項第5款規定，應依職權宣告假執行。原告聲請供擔保准予宣告假執行，僅係促使法院職權之發動，附此敘明。又被告陳明願供擔保請准宣告免為假執行，就原告勝訴部分，經核並無不合，爰依民事訴訟法第第392條第2項規定，酌定相當之擔保金額併准許之；至於原告敗訴部分，其假執行之聲請已失所附麗，應予駁回。七、本件判決之基礎已臻明確，兩造其餘攻擊防禦方法及未經援用之證據，經本院斟酌後，核與判決結果不生影響，爰不另一一論述，併此敘明。八、本件訴訟係原告提起刑事附帶民事訴訟，由本院刑事庭依刑事訴訟法第505條第1項規定移送前來，依同條第2項規定免繳納裁判費，惟本件原告請求系爭機車修復費用及其餘財物損失部分，則非屬前開刑事訴訟之範圍，應另予徵收裁判費1,000元，爰就該部分依民事訴訟法第79條規定，諭知訴訟費用之負擔如主文第2項所示。中華民國109年5月29日民事第一庭法官蕭承信以上正本係照原本作成。如對本判決上訴，須於判決送達後20日內向本院提出上訴狀。如委任律師提起上訴者，應一併繳納上訴審裁判費。中華民國109年5月29日書記官黃進遠"

#on test
#mytext = "臺灣屏東地方法院小額民事判決111年度潮小字第335號原告張孟囷被告倪漢倫上列當事人間請求損害賠償事件，本院於民國111年10月19日言詞辯論終結，判決如下：主文原告之訴駁回。訴訟費用由原告負擔。事實及理由一、原告主張：緣於民國110年11月14日21時6分許，被告駕駛車牌號碼000-0000號自小客車，與訴外人葉吉彥騎乘之車牌號碼000-0000號普通重型機車，於屏東縣內埔鄉建明路與建業路口發生交通事故（下稱系爭事故），造成訴外人即原告配偶王正和所有門牌號碼屏東縣○○鄉○○路00號房屋（下稱系爭房屋）之鋁合金快速捲門、鍛造大門、白鐵烤漆中柱受損。嗣王正和委由原告與被告、葉吉彥聯繫損害賠償和解事宜，被告竟於如附表所示時間，多次以通訊軟體LINE傳送如附表所示留言（下稱系爭留言）予原告，使原告感受到莫大侮辱，被告以言詞辱罵原告，足以貶抑原告人格並使原告受有精神上之痛苦，已侵害原告之人格權。綜上，原告爰依據民法第184條第1項前段、第195條第1項規定提起本訴等語，並聲明：被告應給付原告新臺幣（下同）3萬元，及自起訴狀繕本送達翌日起至清償日止，按年息5％計算之利息。二、被告之答辯：伊固然有於上開時、地，與葉吉彥發生系爭事故之事實，惟葉吉彥確有酒後駕車及未注意車前狀況、減速慢行之疏失，故伊始為附表之系爭留言，嗣伊已與王正和、原告於屏東市調解委員會達成調解，被告已給付和解金額予王正和。又系爭留言僅係伊表述對系爭事故處理過程之想法，並無侮辱原告之意，況系爭留言均為兩造間之私人對話紀錄，並無侵害原告名譽權可言，亦難認原告有何精神上之痛苦，原告請求賠償精神慰撫金，並無理由等語，並聲明：原告之訴駁回。三、本院之判斷：㈠按「因故意或過失，不法侵害他人之權利者，負損害賠償責任。」、「不法侵害他人之身體、健康、名譽、自由、信用、隱私、貞操，或不法侵害其他人格法益而情節重大者，被害人雖非財產上之損害，亦得請求賠償相當之金額。其名譽被侵害者，並得請求回復名譽之適當處分。」，民法第184條第1項前段、第195條第1項分別定有明文。而上開侵權行為之成立，須行為人因故意過失不法侵害他人權利，亦即行為人須具備歸責性、違法性，並不法行為與損害間有因果關係，始能成立。所謂名譽，係指人在社會所享有一切對其品德、聲譽所為之評價，而侵害名譽，係指貶損他人人格在社會上之評價而言，必須依一般社會觀念，足認其人之聲譽已遭貶損始足當之，至於主觀上是否感受到損害，則非認定之標準。又言論自由為人民之基本權利，有實現個人自我、促進民主發展、呈現多元意見、維護人性尊嚴等多重功能，保障言論自由乃促進多元社會正常發展，實現民主社會應有價值，不可或缺之手段。至於名譽權旨在維護個人主體性及人格之完整性，為實現人性尊嚴所必要，二者之重要性固難分軒輊，在法的實現過程中，應力求其二者保障之平衡。故侵害名譽權而應負侵權行為損害賠償責任者，須以行為人意圖散布於眾，故意或過失詆毀他人名譽為必要，蓋如此始有使他人之名譽在社會之評價受到貶損之虞。在一對一之談話中，應賦予個人較大之對話空間，倘行為人基於確信之事實，申論其個人之意見，自不構成侵權行為，以免個人之言論受到過度之箝制，動輒得咎，背離民主社會之本質（最高法院99年度台上字第1664號民事判決意旨參照）。㈡本件被告有於上揭時、地，與葉吉彥發生系爭車禍事故，致系爭房屋受損等情，業經原告提出道路交通事故當事人登記聯單1份附卷為證，且有本院函查之屏東縣政府警察局內埔分局111年8月9日內警交字第11131792700號函文暨所附系爭事故之道路交通事故調查報告表、現場圖、警詢調查筆錄、案發現場照片、道路交通事故初步分析研判表等相關資料在卷可參，且兩造對此並未表示爭執，應堪認屬實。㈢原告主張被告有為系爭留言，已侵害其人格權等語，而被告不否認有為系爭留言之事實，惟為上揭辯解，經查：⑴附表編號1至8部分：原告自承此部分留言均為兩造間一對一之對話（本院卷第79頁背面），則此部分僅為私人間對話，被告所為「噁心」之用語，固然不當，然被告並非公然為之，客觀上應無可能使兩造以外之第三人知悉留言內容，因而致原告之社會評價遭貶損，是此部分留言自難認有何成立侵權行為可言。⑵附表編號9、10部分：原告固陳稱此部分留言係被告於系爭車禍群組所為留言，而車禍群組成員有兩造、葉吉彥及被告保險公司人員等語，被告則辯稱：此部分應該是兩造間之對話等語，經查，原告並未提出其他事證以佐其說，本院尚無從僅憑原告所提出LINE截圖，即認定係被告於車禍群組所為之留言，況觀之此部分留言內容，係被告就系爭車禍事故有關酒駕、肇事責任、保險理賠等事項所為個人意見之陳述，被告固然有部分言詞用語較為激烈，然並未針對原告個人，且亦未虛構不實情事或無端空泛言語謾罵，則參諸上揭說明，被告之個人意見言論表達，應受保障，尚難認有構成侵權行為。至於原告另提出之補充證據2之LINE截圖數張（本院卷第65至73頁），觀其內容應係兩造間就保險理賠事宜所為之相關聯繫內容，自無成立侵權行為可言，一併說明。四、綜上所述，被告所為系爭留言並未成立侵權行為，則原告主張被告之系爭留言有侵害其名譽或居住安寧，依據民法第184條第1項前段、第195條第1項等規定，請求被告賠償精神慰撫金3萬元，及自起訴狀繕本送達翌日起至清償日止，按年息5％計算之利息，為無理由，應予駁回。五、又本件事證已臻明確，兩造其餘陳述及提出之事證，經核與本判決結果不生影響，爰不一一論述指駁，一併說明。六、訴訟費用負擔之依據：民事訴訟法第78條。中華民國111年11月21日潮州簡易庭法官呂憲雄正本係照原本作成。如對判決上訴，須於判決送達後廿日內向本院提出上訴狀，其未表明上訴理由者，應於提出上訴後二十日內向本院補提理由書狀（均須按他造當事人之人數附繕本）。如委任律師提起上訴者，應一併繳納上訴審裁判費。中華民國111年11月22日書記官粘嫦珠附表：編號日期及時間留言內容備註（本院頁數）1111年1月11日7:25噁心72111年1月11日7:26真是噁心83111年1月11日17:54正常人都會覺得這起事件跟機車有關，就你最厲害，說跟機車無關，然後一直騷擾我，然後現在又要跟機車要錢，噁心9、61下4111年2月11日⑴12:18⑵12:23⑴從頭來說，如果機車明知自己沒駕照，又有酒駕可能，依然明知如此且抱持僥倖心態上路，那他就該為自己的行為負責，我當然也會對我行為負責，所以我並沒有說我不願賠償⑵然後你又把這些事實忽略，袒護機車的各種發言，這不噁心？10、61上5111年2月12日8:10一直惡意騷擾的人，還說自己的行為不噁心116111年2月12日8:19很明顯你就是打算恣意濫用法律資源，去獲取不屬於你的利益，不斷騷擾他人，踩到你設的陷阱，惡意的仗著自己是法律背景，知法玩法，並且試圖藉此免除個人公務來請假，做不要必要的法律訴訟127111年2月12日9:26另外，就常理和現在社會觀感來看，你們到底在包庇酒駕肇事行為人什麼？他要賠多賠少，關你什麼事？就是因為你們這種人包庇縱容，哪天葉又酒駕撞死人，你就是幫助殺人犯138110年11月17日17:35不能判定責任後，由機車賠你們嗎？629不詳…這起事件的邏輯很簡單，只要有人有自知之明，無照本來就不該騎機車，要不要和我解釋為何沒駕照嗎？為何知道自己沒駕照還騎機車？為何喝酒還騎機車？不能上路還明知故犯？如果你沒撞到我，什麼事也不會發生，你可以繼續無照酒駕，直到有一天你又弄出人命，台灣就是這樣一堆不會自己行為負責的人，害了別人，還厚顏無恥的不知檢討6310不詳只是我改變想法了，我們看看法院會怎麼判斷，基本上我也不會全責，如果後續是機車全責的話，葉先生你自己考量看看，一開始想說你有受傷，先讓你好好休息，後面打電話我也是和你說好好養傷，後續再談賠償事宜就好，沒關係，結果你打電話過來命令我做事？要提告我刑事，好，我有肇責，刑事判決完的罰金是繳納給國家，你是不會有任何收益的，王夫婦你們最無辜，但是…64"
#mytext = "臺灣彰化地方法院民事判決108年度訴字第1051號原告陳柏村訴訟代理人劉嘉堯律師被告陳淑娥訴訟代理人鄭智文律師複代理人吳文哲律師上列當事人間請求損害賠償事件，本院於民國109年2月25日言詞辯論終結，判決如下：主文被告應給付原告新台幣陸萬陸仟伍佰元，及自民國108年12月10日起至清償日止，按年息5%計算之利息。原告其餘之訴駁回。訴訟費用由被告負擔九分之一，餘由原告負擔。本判決原告勝訴部分得假執行；但被告如以新臺幣陸萬陸仟伍佰元供擔保後，得免為假執行。事實及理由一、原告起訴主張：（一）被告於民國（下同）107年11月17日9時23分許，駕駛車牌號碼000-000號普通重型機車，沿彰化縣永靖鄉浮圳路由北往南行駛，行經浮圳路與浮圳路693巷口，右轉浮圳路693巷時，本應行經無號誌交叉路口，左方車應暫停讓右方車先行，且當時柏油路面乾燥無缺陷等無不能注意之情事，竟疏未注意應暫停讓右方車先行，適有原告騎乘車牌號碼000-000普通重型機車沿浮圳路693巷由西往東行駛至該處，發生撞擊，致原告受有左側手肘挫傷併扭傷等傷害，並經本院108年度交簡字第860號刑事簡易判決被告過失傷害拘役55日在案。依彰化縣區車輛行車事故鑑定會鑑定意見書可知，被告駕駛普通重型機車，行經無號誌交叉路口，左方車未暫停讓右方車先行，為本件事故之肇事主因，應負擔較高之過失責任。原告駕駛普通重型機車，行經無號誌交叉路口，疏未減速慢行，做隨時停車之準備，為肇事次因。因本件事故發生，致原告受有上開傷害，原告事後經員林基督教醫院及沙鹿童綜合醫院診斷，左手韌帶移位建議開刀治療，減少工作收入，且身心受極大精神痛苦，爰依民法第184條第1項前段、第2項、第191條之2、第193條第1項、第195條第1項前段等規定，請求被告賠償損害。（二）原告對被告請求之項目、金額如下：1、減少工作收入：原告自本件車禍發生至入監執行前，共四個月因左手傷勢需休養無法工作，造成原先養雞、割筍、割香蕉等月入新台幣（下同）25000元之勞力工作收入減少，受有十萬元之損害。2、精神慰撫金：本件車禍造成原告相當之精神上痛苦，故請求賠償精神慰撫金70萬元。（三）綜上所述，原告請求金額共80萬元，然因被告應負擔百分之七十過失責任，故原告請求被告賠償56萬元。並聲明：⑴被告應賠償原告56萬元，並自起訴狀繕本送達之翌日起至清償日止，按年利率百分之五計算之利息。二、被告則略以：（一）按當事人主張有利於己之事實者，就其事實有舉證之責任。民事訴訟法第277條本文定有明文。次按當事人於其利己事實之主張，除事實於法院已顯著，或為其職權上已認知者外，應負立證之責。負舉證責任之當事人，須證明至使法院就該待證事實獲得確實之心證，始盡其證明責任。民事訴訟如係由原告主張權利者，應先由原告負舉證之責，若原告先不能舉證，以證實自己主張之事實為真實，則被告就其抗辯事實即令不能舉證，或其所舉證據尚有疵累，亦應駁回原告之請求。負舉證責任之當事人，須證明至使法院就該待證事實獲得確實之心證，始盡其證明責任。倘不負舉證責任之他造當事人，就同一待證事實已證明間接事實，而該間接事實依經驗法則為判斷，與待證事實之不存在可認有因果關係，足以動搖法院原已形成之心證者，將因該他造當事人所提出之反證，使待證事實回復至真偽不明之狀態。此際，自仍應由主張該事實存在之一造當事人舉證證明之，始得謂已盡其證明責任，否則即應就該事實真偽不明之狀態承擔此一不利益。最高法院106年度台上字第2062號、第2867號、99年度台上字第1582號判決參照。（二）復按侵權行為之成立，應具備：（1）須該他人有加害行為（2）須侵害權利或利益3）須發生損害（4）須加害行為與損害有因果關係（5）須有責任能力（6）須有故意或過失等六項要件。若主張他人應負侵權行為之損害賠償責任，除有舉證責任倒置之規定外，應先就上開6項要件負完全之舉證責任。再按加害行為與損害有因果關係之要件係指二者之間，有相當因果關係為成立要件。故上訴人所主張損害賠償之債，如不合於此項成立要件者，即難謂有損害賠償請求權存在（最高法院四十八年台上字第四八一號判例意旨參照）。至所謂相當因果關係，係以行為人之行為所造成之客觀存在事實，依經驗法則，可認通常均可能發生同樣損害之結果而言；如有此同一條件存在，通常不必皆發生此損害之結果，則該條件與結果並不相當，即無相當因果關係；不能僅以行為人就其行為有故意過失，即認該行為與損害間有相當因果關係（最高法院九十年度台上字第七七二號判決意旨參照）。是侵權行為損害賠償請求權人得向侵權行為人請求之範圍，必以所受損害，與責任原因事實間，依其情形均可發生同樣之損害，始堪肯認在得請求賠償之列。臺灣高等法院臺中分院103年度上字第517號及福建高等法院金門分院104年度上易字第2號判決參照。（三）本件原告固主張：伊因被告陳淑娥之行為受有左側手肘挫傷併扭傷等傷害，致有減少工作收入10萬元之損害，並請求精神慰撫金70萬元云云。然原告主張「減少工作收入10萬元」，並無證據資料可資證明有該損害及與被告行為有何因果關係？另鈞院刑事庭108年5月27日108年度交簡字第860號判決亦認原告所受傷勢尚非嚴重等語，是原告請求精神慰撫金70萬元顯然過高。再者，原告108年10月2日民事起訴暨聲請訴訟救助狀所附童綜合醫療社團法人童綜合醫院一般診斷書，載明門診治療日期為107（2018）年12月4日，而本件原告主張事故時間乃107年11月17日，相距17日，則該左手肘側韌帶損傷，是否為本件被告行為所致？並非無疑。從而，被告否認原告起訴狀所載涉及「減少工作收入10萬元」、「精神慰撫金70萬元」，倘原告認於其有利，揆以前開法條規定、實務見解，應由原告就減少工作收入10萬元等損害，及該等損害與被告行為間有相當因果關係等有利於原告之事實負舉證之責。（四）聲明：（1）原告之訴及假執行之聲請均駁回。（2）如受不利判決，願供現金或金融機構可轉讓定期存單擔保，請准宣告免為假執行。三、本院得心證之理由：（一）原告主張其因被告於上開時間、地點，疏未注意左方車未暫停讓右方車先行之過失行為，致原告受有前開損害，業據其提出診斷書等為證，且被告因上開過失行為經本院108年度交簡字第860號刑事簡易判決判處拘役55日，如易科罰金，以1,000元折算1日，有該刑事判決書在卷可稽，並經本院調閱上開刑事卷宗全卷查明無誤。又被告對於前開判決所認定之事實不爭執，足見被告對本件車禍之發生顯有過失，且因被告之過失不法行為與原告所受傷害之結果間，具有相當因果關係，是原告依侵權行為法律關係請求被告賠償其因本件車禍所受之損害，自屬有據。然被告辯稱對於不能工作之損失部分，原告並未提出相關證據證明，另以原告所受傷害情形觀之，原告請求之精神慰撫金70萬元云云，顯然過高，請求駁回原告之訴。（二）按因故意或過失，不法侵害他人之權利者，負損害賠償責任。違反保護他人之法律，致生損害於他人者，負賠償責任。汽車、機車或其他非依軌道行駛之動力車輛，在使用中加損害於他人者，駕駛人應賠償因此所生之損害。又不法侵害他人之身體或健康者，對於被害人因此喪失或減少勞動能力或增加生活上之需要時，應負損害賠償責任；又不法侵害他人之身體、健康、名譽、自由、信用、隱私、貞操，或不法侵害其他人格法益而情節重大者，被害人雖非財產上之損害，亦得請求賠償相當之金額，民法第184條第1、2項、第191條之2、第193第1條、第195條第1項分別定有明文。經查，被告對原告請求之數額有爭執，是本件所應探究者，厥為本件原告得向被告請求之損害賠償金額為何？1、減少工作收入：原告稱自本件車禍發生至其入監執行前，共四個月因左手傷勢需休養無法工作，造成原先養雞、割筍、割香蕉等月入新台幣（下同）25000元之勞力工作收入減少，受有十萬元之損害云云。經查，原告108年10月2日民事起訴暨聲請訴訟救助狀所附童綜合醫療社團法人童綜合醫院一般診斷書，載明門診治療日期為107（2018）年12月4日，而本件事故時間為107年11月17日，相距僅17日，而無其他獨立外力原因所中斷，可認原告之左手肘側韌帶損傷，為本件被告上開過失行為所致。原告主張四個月無法工作，然因此診斷書所載，醫師囑咐原告宜休養六周（42天），而原告63年次，有年級資料可參，平時養雞及在種植香蕉農作物，以其月入25000元計算，42天減少之工作收入為35000元，是原告請求工作損失35000元，尚屬合理，自應准許；其餘原告逾越此範圍之請求，自無足採，應予駁回。2、精神慰撫金：按法院於酌定慰撫金數額時，應斟酌加害人與被害人雙方之身分、資力與加害程度，及其他各種情形核定之（最高法院51年台上字第223號判例意旨參照）。查本件原告因本件車禍受有左手韌帶損傷，經員林基督教醫院及沙鹿童綜合醫院診斷，事後經童綜合醫院診斷左手韌帶斷裂建議開刀治療，身心受極大精神痛苦，本院審酌審酌兩造之資力，原告名下無財產，被告無工作等情，考量原告之傷勢及兩造之社會經濟地位，故認原告請求精神慰撫金70萬元，顯然過高，應核減為6萬元方屬適當，原告逾此範圍之請求，則屬過高，應予駁回。3、綜上所述，原告可請求之費用共計95000元【計算式：35000元＋60000元＝95000元】。（三）按損害之發生或擴大，被害人與有過失者，法院得減輕賠償金額，或免除之。民法第217條第1項定有明文。經查，依彰化縣區車輛行車事故鑑定會鑑定意見書可知，被告駕駛普通重型機車，行經無號誌交叉路口，左方車未暫停讓右方車先行，為本件事故之肇事主因。原告駕駛普通重型機車，行經無號誌交叉路口，疏未減速慢行，做隨時停車之準備，為肇事次因，此有鑑定意見書可稽，本院認為被告為本件事故之肇事主因，應負擔十分之七之過失責任，原告就車禍之發生亦有過失，應負擔十分之三過失責任失責任。依此過失比例計算，原告可請求之費用為66,500元【計算式：95,000元×70%＝66,500元】，故原告僅得請求被告賠償66500元。（四）綜上所述，原告本於侵權行為規定，請求被告賠償原告66,500元，為屬正當，應予准許，其餘逾此數額之請求，則非有據，應予駁回。又原告併請求自起訴狀繕本送達翌日即108年12月10日起至清償日止之法定遲延利息，核無不合，併予准許。四、原告勝訴部份，並未超過50萬元，合於民事訴訟法第389條第1項第5款規定，本院應依職權宣告假執行：另依據被告聲請，定相當擔保金額准被告供擔保後得免為假執行。五、結論：本件原告之訴，為一部有理由，一部無理由，並依民事訴訟法第79條、第389條第1項第5款，第392條，判決如主文。中華民國109年3月13日民事第一庭法官施坤樹以上正本係照原本作成。如對判決上訴，應於判決送達後20日內向本院提出上訴狀。如委任律師提起上訴者，應一併繳納上訴審裁判費。中華民國109年3月23日書記官詹國立"

#Incorrect
#特別的Case，因為有「請求精神慰撫金120萬元，被告則主張過高，應以10萬元為適當等語。」所以模型以為這句話是法官說的，因此要把這種特殊case的被告反駁的金額也label起來
#mytext = "臺灣臺中地方法院民事判決104年度訴字第2131號原告林怡璇訴訟代理人施學源被告邱凡容（原名邱逢任）訴訟代理人簡毓森上列被告因過失傷害案件（本院103年度交易字第1238號），經原告提起刑事附帶民事訴訟請求損害賠償，經本院刑事庭裁定移送前來（本院103年度交附民第237號），於民國104年12月22日言詞辯論終結，本院判決如下：主文被告應給付原告新臺幣肆拾萬柒仟柒佰伍拾壹元。原告其餘之訴駁回。本判決第一項得假執行。訴訟費用由被告負擔十分之一，餘由原告負擔。原告其餘假執行之聲請駁回。事實及理由一、原告主張：㈠、被告於民國102年11月7日下午3時55分許，駕駛車牌號碼000-0000號自用小客車，沿臺中市沙鹿區臺灣大道6段由南往北方向行駛，途經臺灣大道6段與正英路交岔路口，正欲左轉正英路時，本應注意車前狀況，隨時採取必要之安全措施，且行車時應遵守交通號誌，而依當時情形天氣晴朗、日間光線充足，因而受有左膝前十字韌帶創傷性破裂、左膝後十字韌帶創傷性破裂等傷害。㈡、對於被告上開傷害之行為，原告依民法侵權行為之法律關係，請求被告負損害賠償責任。原告請求之損害賠償金額詳細項目如下：⒈醫療費用：原告醫療費用自付金額新臺幣（下同）18萬元。⒉機車修理費：58,000元。⒊薪資損失：原告於臺中市中工三路食儀天韻公司打工，因本件車禍受傷，請求被告賠償一年薪資損失288,000元。⒋交通費：原告發生車禍後，由原告父親每天載原告上下學及去醫院，請求一年交通費18萬元。⒌看護費用：自發生車禍開始，前半年專人看護36萬元，後半年一般看護18萬元，共計54萬元。⒍中藥加新鮮鱸魚：96,000元。⒎喪失勞動能力：原告因車禍功能喪失3至4成，影響勞動損失補貼172萬元。⒏後續醫療費用：原告需醫學美容，人工韌帶可能再復發，可能還要再開刀一次，請求後續醫療保障金96萬元。⒐精神慰撫金：原告肌肉萎縮，造成永久創傷，請求精神慰撫金120萬元。㈢、並聲明：⑴被告應給付原告522萬元。⑵願供擔保請准宣告假執行。㈣、對被告抗辯之陳述：原告有收受被告12萬元之賠償金及汽機車強制責任保險金69,515元，原告同意自請求之金額中扣除。另本件事故之發生原告無過失，是被告闖紅燈。二、被告抗辯：㈠、對於原告請求醫療費用自付金額，就原告提出榮總醫院之醫療費用收據部分、膝關節支架8,500元、人工皮及凝膠10,800元部分不爭執，其餘部分爭執。機車修理費58,000元不爭執，但零件部分需折舊。原告請求看護費部分，被告只同意賠償全日看護三個月，一天2,000元計算，總計18萬元。原告請求功能喪失三、四成勞動補貼172萬元，經榮總鑑定結果並無勞動減損情形。原告主張醫學美容、人工韌帶及醫療保障金96萬元，請原告舉證。關於原告主張之薪資損失288,000元、一年交通費18萬元、中藥加新鮮鱸魚96,000元部分，被告均不同意，被告否認食儀天韻公司出具之證明書之真正。精神慰撫金部分，被告主張以10萬元為適當。㈡、被告前已賠償原告12萬元，另原告已領取汽機車強制責任保險金69,515元，被告主張依法應予扣除。㈢、並聲明：⑴原告之訴及假執行之聲請均駁回。⑵如受不利判決，請准供擔保免為假執行。三、兩造同意爭執及不爭執事項如下：㈠、不爭執事項⒈被告於102年11月7日下午3時55分許，駕駛車牌號碼000-0000號自用小客車，沿臺中市沙鹿區臺灣大道6段由南往北方向行駛，途經臺灣大道6段與正英路交岔路口，正欲左轉正英路時，本應注意車前狀況，隨時採取必要之安全措施，且行車時應遵守交通號誌，而依當時情形天氣晴朗、日間光線充足、路面狀況良好、無缺陷及障礙、視距良好，並無不能注意之情事，竟未待左轉箭頭綠燈時相亦沿臺灣大道6段由北往南方向直行，行經上開路口時，即與被告駕駛之前開自用小客車左側車身發生碰撞，致原告人車倒地，因而受有左膝前十字韌帶創傷性破裂、左膝後十字韌帶創傷性破裂等傷害，被告所涉過失傷害犯行，經本院103年度交易字第1238號判處有期徒刑4月，得易科罰金確定。⒉原告於本件車禍並無過失。⒊原告因本件車禍受傷支出膝關節支架8500元，人工皮及凝膠10800元。⒋車牌號碼000-000號為原告所有，於102年8月出廠。⒌被告前已賠償原告12萬元。⒍原告已領取汽機車強制責任保險金69,515元。一年交通費18萬、看護費用54萬、中藥加新鮮鱸魚96000元、喪失勞動能力172萬元、後續醫療費用96萬元、精神慰撫金120萬元，有無理由？數額應為何？四、得心證之理由㈠、原告主張被告於前揭時間，駕駛上開自用小客車，沿臺中市沙鹿區臺灣大道6段由南往北方向行駛，途經臺灣大道6段與正英路交岔路口，正欲左轉正英路時，本應注意車前狀況，隨時採取必要之安全措施，且行車時應遵守交通號誌，而依當時情形並無不能注意之情事，未待左轉箭頭綠燈時相，於直行綠燈時即貿然闖越紅燈左轉，且未注意車前狀況，適有原告騎乘上開機車，亦沿臺灣大道6段由北往南方向直行，行經上開路口時，即與被告駕駛之前開自用小客車左側車身發生碰撞，致原告人車倒地，因而受有左膝前十字韌帶創傷性破裂、左膝後十字韌帶創傷性破裂等傷害之事實，為被告所不爭執，而被告所涉過失傷害犯行，並經本院103年度交易字第第1238號判處有期徒刑4月，得易科罰金確定，亦有本院調職上開卷宗核閱無訛。被告駕車行經前揭交岔路口，疏未注意應遵守燈光號誌及車前狀況，而貿然闖越紅燈左轉，致與原告機車發生碰撞，原告因而受有上述傷害，顯違反交通安全規則第102條第1項第1款、第94條之規定，自屬有過失，而被告之過失行為與原告之傷害結果間，確有相當因果關係存在，亦堪認定。㈡、按因故意或過失，不法侵害他人之權利者，負損害賠償責任；而不法侵害他人之身體或健康者，對於被害人因此喪失或減少勞動能力或增加生活上之需要時，應負損害賠償責任；不法侵害他人之身體或不法侵害其他人格法益而情節重大者，被害人雖非財產上之損害，亦得請求賠償相當之金額，民法第184條第1項前段、第193條第1項、第195條第1項分別定有明文。查被告因前揭過失行為致原告受有前述傷害，原告依據上開法律規定，向被告請求侵權行為之損害賠償，即屬有據。茲就原告請求被告給付損害賠償金額內容加以審酌如下：⑴、醫療費用支出：原告主張其因本件車禍支出醫療費用18萬元，據其提出榮總住院、門診醫醫療收據、創健國際有限公司統一發票及統一發票6紙，被告則對原告提出榮總之醫療費用收據金額、關節支架8500元、及關於倍舒痕凝膠、德國人工皮之4張統一發票不爭執。依據原告提出其在榮總之住院、門診醫療收據之數額共計98,256元【95231元+500元+480元+100元+160元+100元+110元+480+480元+215元+100元+100元+200元=98,256元，見本院卷第15至18頁、22至29頁】，原告另主張其支出關節支架、倍舒痕凝膠、德國人工皮部分共計19300元【8500元+3000元+3000元+1800元+3000元=19,300元，見本院卷第19頁、35頁】，為原告醫療之必要支出，亦為被告所不爭，應予准許。至原告提出合利他命4000元部分，原告並未證明其醫療之必要性為何，自不應准許，另強補鈣3600元部分，按鈣片所含之維生素D等營養素，自一般飲食中即可攝取，且原告亦未證明其強補鈣支出部分為其醫療之所必要，且為被告所否認，自不應准許。是以本件原告請求醫療費用支出117,556元【98,256元+19,300元=117,556元】為有理由，逾此部分，即不予准許。⑵、機車修理費用：原告主張其因本件車禍，致原告機車損害，修理費用58,000元，被告應負賠償責任，業據其提出信竑機車行收據為據，被告對於機車修理費用數額並不爭執，惟主張應計算折舊等語。按不法毀損他人之物者，應向被害人賠償其物因毀損所減少之價額，民法第196條定有明文。故物被不法毀損後，僅須其物之價額減少，即須賠償其所減少之價額。又依民法第196條請求賠償物被毀損所減少之價額，得以修復費用為估定之標準，但以必要者為限（例如：修理材料以新品換舊品，應予折舊）。被害人如能證明其物因毀損所減少之價額，超過必要之修復費用時，就其差額，仍得請求賠償（最高法院77年度第9次民事庭會議決議參照）。本件原告機車之零件修理既係以新零件更換被損壞之舊零件，則原告以修理費作為損害賠償之依據，自應將零件折舊部分予以扣除。查，本件依據原告提出之信竤機車行收據記載：零件一批，58000元（見本院卷第31頁），為被告所不爭。又折舊標準應依行政院所頒固定資產耐用年數表及固定資產折舊率之規定，機器腳踏車之耐用年數為3年，依定率遞減法每年折舊千分之536。原告所有之機車為102年8月出廠，有交通部公路總局臺中區監理所臺中監理站104年11月26日中監中站字第0000000000號函檢送車籍資料在卷可稽（見本院卷第117至118頁），距本件車禍發生之102年11月7日約3.2個月，依上開說明，以定率遞減法每年折舊千分之536計算，機車修理費用為58,000元，扣除折舊後原告所得請求之修理費用為49,710元【計算式：58000－（58,000536/1000×3.2/12）＝49710元以下四捨五入】，因此本件原告得請求之機車修理費用為49,710元，其逾此金額之請求則屬無據。⑶、薪資損失：原告主張其臺中市中工三路之食儀天韻公司打工，大約做了三個多月就發生車禍，薪資是支付現金，請求被告賠償其一年薪資損失288,000元，惟為被告所否認。本件固據原告提出食儀天韻有限公司出具之留職停薪證明記載：「因員工林怡璇車禍受傷，治療後仍無法站及膝蓋無法彎曲，已嚴重影響工作運作，本店將予留職停薪，停薪日於102年11月7日至103年11月7日止，工作期間底薪24000元，獎金加給0000-0000」等語，惟被告否認上開文書之真正。本院為此發函詢問食儀天韻有限公司關於原告於該公司之工作期間、工作性質及內容為何，並請其檢附任職期間之扣繳憑單及出勤記錄供參，惟均未據食儀天韻有限公司回覆，另本院調取原告勞、健保投保資料及稅務電子閘門資料，原告僅有103年6月以後在來來超商打工（部分工時）之投保及所得資料，在103年6月以前並無任何因工作而投保及所得之證明，原告亦未提出任何自食儀天韻公司支領薪資之證明，則僅憑原告提出食儀天韻有限公司以手寫之留職停薪證明書（被告否認其真正），自難信原告主張其確有於該公司工作並領取每月24000元薪資之事實為真。另參以原告於車禍發生時為弘光技術學院一年級在校學生，業據原告陳明在卷，而原告既未能提出原告當時確實有受僱而領取薪資之證明，自難逕認其受有何薪資之損失。是以原告請求被告賠償其1年之薪資損失288,000元，為屬無據，不應准許。⑷、交通費用：原告主張其發生車禍後，由原告父親每天載原告上下學及去醫院，請求一年交通費18萬元，固據其提出施學源出具之收據乙紙其上記載「車資包月1個月15000元，計12個月」。惟查施學源乃原告之父親即本件訴訟代理人，惟衡以原告與施學源為父女關係，原告是否確實支付其父親一年之車資已不無疑問，原告既未提出其父親施學源以開車為業，且確實有支付其父親18萬元車資之證明，自難僅憑施學源出具之收據即逕認原告確有支出18萬元之交通費用。況且，依據收據內容記載亦無從得知是否為原告因本件車禍受傷就醫所生之交通費用，是本件原告請求被告賠償18萬元之交通費用，核屬無據。⑸、看護費用：原告主張自發生本件車禍開始，前半年專人看護36萬元，後半年一般看護18萬元，總計54萬元，業據其提出林秀蘭出具之免用發票收據為證，則為被告所否認。本件據臺中榮民總醫院104年10月7日中榮醫企字第0000000000號函覆本院說明：依原告受傷狀況，需專人全日看護3個月尚合乎目前民間看護行情，並為被告所不爭。是以本件原告可請求之看護費用應為18萬元【計算式：90日X2000元=18萬元】，逾此範圍之請求，則屬無據。⑹、原告請求被告賠償中藥加新鮮鱸魚費用96,000元，為被告所否認。本件原告既未舉證證明中藥加新鮮鱸魚為原告醫療之所必要，自難逕予准許。⑺、喪失勞動能力：原告主張其因車禍功能喪失3至4成，影響勞動損失補貼172萬元，為被告所否認。本件車禍案件於本院刑事庭審理時，經承審法官送臺中榮民總醫院鑑定結果認：「原告於102年11月7日因交通事故導致左膝十字韌帶斷裂，於本院接受韌帶重建手術。於104年4月17日鑑定其健康情況如下：右膝關節活動度130度、左膝關節活動度120度。左大腿肌肉萎縮，右大腿腿圍49公分、左大腿腿圍46公分。可獨立行走。原告雖有左大腿肌肉萎縮，但左膝關節活動度並未受限達正常活動度三分之一以上，不符合勞保失能給付標準所規定之失能條件，故無勞動能力減損情形」則本件原告既無勞動能力減損之情形，其請求被告賠償勞動能力損失172萬元，為屬無據。人工韌帶可能再復發，請求後續醫療保障金96萬元，為被告所否認。本件經本院函詢臺中榮民總醫院關於原告是否有進行第二次手術之必要，預計之醫療費用支出為何，而據該院104年10月7日中榮醫企字第0000000000號函說明：「是否有進行第二次手術之必要，目前無法判斷」（見本院卷第94頁），則原告既未能證明其以後有再進行手術之必要，則其請求後續醫療費用保障金96萬元，為屬無據。⑼、精神慰撫金：原告主張因被告過失致其肌肉萎縮，造成永久創傷，請求精神慰撫金120萬元，被告則主張過高，應以10萬元為適當等語。按慰撫金之賠償須以人格權遭遇侵害，使精神上受有痛苦為必要，其核給之標準固與財產上損害之計算不同，然非不可斟酌雙方身分資力與加害程度，及其他各種情形核定相當之數額。又按以人格權遭遇侵害，受有精神上之痛苦，而請求慰藉金之賠償，其核給之標準，須斟酌雙方之身分、資力與加害程度及其他各種情形核定相當之數額。且所謂「相當」，應以實際加害情形與其名譽影響是否重大及被害人之身分、地位與加害人之經濟情況等關係定之（最高法院51年台上字第223號判例、86年度台上字第3537號判決要旨參照）。原告因本件車禍受有左膝前十字韌帶創傷性破裂、左膝後十字韌帶創傷性破裂等傷害，造成身體之痛苦及生活之不便，難謂無精神上之痛苦。再查原告為大學在校學生，名下無汽車或不動產，被告為技術學校肄業，於私人公司擔任業務代表，薪資所得每月3萬元，名下有自用住宅一戶等情，業據兩造陳明在卷，並有本院調取兩造稅務電子閘門財產調件明細表在卷可參。本院審酌原告受有上開傷害所受痛苦之程度、期間，及兩造社會身分、地位、財產經濟狀況、工作與收入等事實，因認原告得請求被告賠償精神慰撫金25萬元為屬適當，逾此範圍，則不予准許。"

#特別的Case 2，因為是「上訴人」開頭，在我的樣本裡面沒有讓他學過，所以模型不知道誰是原告，整體也學得不好。
#mytext = "臺灣苗栗地方法院民事判決110年度簡上字第18號上訴人陳彥宏即聖宏實業社被上訴人A01（真實姓名年籍資料詳卷）法定代理人甲○○（真實姓名年籍資料詳卷）上列當事人間請求侵權行為損害賠償事件，上訴人對於民國110年2月26日本院苗栗簡易庭110年度苗簡字第61號第一審判決提起上訴，本院於110年7月28日言詞辯論終結，判決如下：主文上訴駁回。第二審訴訟費用由上訴人負擔。事實及理由甲、程序方面：按在第二審為訴之變更或追加，非經他造同意，不得為之，但請求之基礎事實同一、擴張應受判決事項之聲明者，不在此限，民事訴訟法第446條第1項但書、第255條第1項第2、3款定有明文。上訴人即反訴原告（下稱上訴人）於原審反訴請求被上訴人給付新臺幣（下同）18,266元，經原審駁回其請求，上訴人提起上訴，並於本院追加反訴請求為98,266元，經核係屬於基於同一車禍事故所為聲明之擴張，於法尚無不合，應予准許，合先敘明。乙、實體方面：壹、本訴部分一、被上訴人起訴主張：上訴人於民國108年4月12日下午3時許，駕駛車牌號碼000-0000號營業用小貨車（下稱A車），沿苗栗縣○○鎮○○里0鄰0○0號旁南北向產業道路（下稱甲路），由南往北行駛至該路與東西向產業道路（下稱乙路）交岔路口時，上訴人為支線道車未禮讓幹線道車先行，即貿然左轉，適有訴外人乙○○騎乘車牌號碼000-0000號普通重型機車（下稱B車）搭載被上訴人沿乙路由東往西駛至案發地點，致閃避不及兩車發生碰撞，B車倒地（下稱系爭事故），被上訴人因而受有臉部、四肢多處擦挫傷、左膝嚴重挫傷疑韌帶受損等傷害（下稱系爭傷害），為此支出必要之醫療費用14,939元，且被上訴人因受有本案傷害，心理受創致生失眠症狀，精神上遭受極大痛苦，爰依民法第184條第1項前段、第191之2條、第193條第1項、第195條第1項規定，提起本件訴訟，求為判決：上訴人應給付被上訴人814,939元，及自起訴狀繕本送達翌日起至清償日止，按年息5%計算之利息。二、上訴人則以：否認碰撞地點位於轉彎處，由A車鈑金毀損情形，可知系爭事故發生並非兩造垂直對撞所致，係B車由後方飆速超越A車欲左轉，B車把手碰撞到A車右大燈上方鈑金所造成，因機車欲轉彎時會與地面呈現15度的斜度，經其衝撞後造成A車鈑金凸出，右大燈下方並未受損實屬正常，若上訴人係於交岔路口左轉時與B車碰撞，A車車牌以下之鈑金定會受損嚴重，且遭撞後乙○○及被上訴人應跌落於A車右方，而非分別跌落於A車右前方1公尺處及A車正前方，若依乙○○所述，兩車係於路口處碰撞，則應是B車前輪碰撞A車右下方鈑金，但A車右下方鈑金全無碰撞痕跡。乙○○無照搭載被上訴人並飆車蛇行，嚴重影響用路人安全，始致系爭事故發生，上訴人並無過失等語，茲為抗辯。貳、反訴部分一、上訴人提起及於本院追加反訴主張：系爭事故造成A車受損，致上訴人受有修車費用18,266元之損害，且本案純屬輕微碰撞事件，被上訴人明知系爭事故是因其飆車蛇行所致，卻以不實理由提告，浪費司法資源，造成上訴人兩年多往返法院30餘次，故追加請求被上訴人給付工作損失8萬元，被上訴人理該知道乙○○為危險無照駕駛，應予制止，有責任分擔之必要等語，爰依侵權行為之法律關係，訴請被上訴人賠償上開損害。並聲明：被上訴人應給付上訴人98,266元。二、被上訴人則以：上訴人未因系爭事故受有體傷，請求工作損失並無理由，另A車受損係因乙○○過失所致，與被上訴人是否搭乘乙○○之B車無涉，故上訴人請求修車費用亦屬無理由等語。參、原審就本訴部分為被上訴人部分勝訴之判決，判命上訴人給付被上訴人52,963元及自109年5月1日起至清償日止，按年息5％計算之利息，並駁回其餘之訴，被上訴人就其敗訴部分未聲明上訴已告確定，上訴人就其敗訴部分提起上訴；另原審就原反訴請求為上訴人敗訴之判決，上訴人提起上訴，並於本院追加反訴請求工作損失，並聲明：㈠本訴部分：原判決不利於上訴人部分廢棄；上開廢棄部分，被上訴人在第一審之訴駁回。㈡反訴部分：原判決廢棄，被上訴人應給付上訴人98,266元。被上訴人則答辯聲明：上訴駁回。肆、本院協同兩造整理不爭執之事實如下：㈠上訴人於案發時間，駕駛A車，行至案發地點時，適有乙○○駕駛B車搭載被上訴人駛至，致使系爭事故發生，被上訴人因而受有系爭傷害。㈡甲路鄰近與乙路之交岔路口處地面有繪製「停」標字，甲路屬支線道，乙路屬幹線道。㈢上訴人因系爭事故經本院刑事庭以109年度交易字第77號判處有期徒刑3月，如易科罰金，以1,000元折算1日，上訴人提起上訴，經臺灣高等法院臺中分院以109年度上易字第1051號判決駁回上訴確定（下稱刑案）。㈣被上訴人因本件車禍受傷至醫療院所接受治療支出醫療費用共計14,939元。㈤系爭事故發生時，搭載被上訴人之乙○○並未領有普通重型機車駕駛執照。㈥被上訴人已領有上訴人強制險投保公司明台產險之強制險給付16,724元。㈦被上訴人已與乙○○和解，並有收受2萬元之賠償金。㈧上訴人為修復A車因本件車禍事故所受之損害預估須支出鈑金550元、烤漆6,058元、零件9,228元、工資2,430元，共計18,266元。伍、得心證之理由：一、本訴部分：㈠上訴人就系爭事故之發生具有過失，應對被上訴人負侵權行為損害賠償責任：⒈按因故意或過失不法侵害他人之權利者，負損害賠償責任；汽車、機車或其他非依軌道行駛之動力車輛，在使用中加損害於他人者，駕駛人應賠償因此所生之損害，但於防止損害之發生，已盡相當之注意者，不在此限，民法第184條第1項前段、第191條之2分別定有明文。又不法侵害他人之身體、健康，被害人雖非財產上之損害，亦得請求賠償相當之金額，民法第195條第1項定有明文。又停車再開標誌「遵１」，用以告示車輛駕駛人必須停車觀察，認為安全時，方得再開。設於安全停車視距不足之交岔道路次要道路口；「停」標字，用以指示車輛至此必須停車再開。設於停止線將近之處，本標字與第58條「停車再開」標誌得同時設置或擇一設置；汽車行駛至交岔路口，其行進、轉彎，應依下列規定：二、行至無號誌之交岔路口，支線道車應暫停讓幹線道車先行，道路交通標誌標線號誌設置規則第58條第1項、第177條第1項、道路交通安全規則第102條第1項第2款亦有明文。⒉經查，車禍現場為交叉路口，而甲路路口處之路面設有「停」字標線，有刑案卷附道路交通事故現場圖在卷可證（見警卷第19頁），足認於案發路口處甲路應屬支線道，乙路則為幹線道。又查，乙○○於偵訊中具結證述：伊騎車沿乙路直行，在路口處A車突然衝出來，我們雙方車輛就發生碰撞，在快要發生碰撞之前，伊看路口反光鏡有看到A車，當時伊以為他會停下來，結果他左轉衝出來，我們就發生碰撞，碰撞當時A車車頭才剛轉出來，尚未完全轉入乙路之中，等語（見偵卷第18頁）；觀之二車於系爭車禍後，A車之右前車燈、保險桿損壞、B車之左側車身損壞，經有刑案卷附車損照片及上訴人提供之A車照片在卷可憑（見警卷第25至26頁，本院卷第41頁），若A車由南往北駛至案發路口，在左轉欲進入乙路時恰與由東向西駛至之B車發生碰撞，則雙方車輛之碰撞點方即有可能係位在A車右前方，即A車上開車損處，益見乙○○前開結證內容應屬實情，堪以採認。是以，上訴人既行駛於支線道之甲路，其於進入路口前，如有謹慎注意幹線道即乙路來車即B車並讓其先行，無甚可能與B車相撞，查依刑案所附道路交通事故調查報告表㈡之記載，上訴人考領有駕駛執照，對此規定應係知之甚詳，且因當時天候晴、日間自然光線、柏油路面乾燥、無缺陷、無障礙物且視距良好，並無不能注意之情事，惟上訴人疏於注意，未確實注意幹線道上有無來車並讓其先行，即貿然自支線道左轉欲進入幹線道而致生系爭事故，顯有過失。又系爭事故前經送交通部公路總局新竹區監理所竹苗區車輛行車事故鑑定會進行鑑定，並經交通部公路總局車輛行車事故鑑定覆議會進行覆議後，鑑定及覆議結果均認：上訴人駕駛自用小貨車，行經劃設「停」標字之無號誌路口左轉彎，支線車未讓幹道車先行，為肇事主因；乙○○駕駛普通重型機車，行經無號誌交岔路口，未減速慢行，作隨時停車之準備，為肇事次因等情，有刑案卷附車輛行車事故鑑定會鑑定意見書及鑑定覆議會覆議意見書各1份在卷可參（見偵卷第63至66頁，刑案一審卷第81至83頁），益見上訴人就系爭事故確有過失甚明。再被上訴人因上訴人之過失駕車行為，致受有系爭傷害，並於案發當日前往醫院就診，有診斷證明書1紙在卷可證（見偵卷第29頁），李綜合醫療社團法人苑裡李綜合醫院函文並說明被上訴人左膝嚴重挫傷疑韌帶受損亦係因系爭事故所致（見原審卷第181頁），足認上訴人之過失駕駛行為，確與被上訴人所受系爭傷害結果間具有相當因果關係，故被上訴人依得上開規定向上訴人請求侵權行為損害賠償。⒊至上訴人雖稱：A車當時已完全轉入乙路，乙○○所騎B車才自後追撞A車，如係路口處發生碰撞，乙○○騎乘B車應該會是前輪與其貨車發生碰撞，為何會在B車左把手，且其所駕A車前方板金無碰撞痕跡、右前車頭板金係向前突出，可見被上訴人跟乙○○都說謊云云，並聲請送請學術單位鑑定及測謊。然查，B車如係由後追撞已完全轉入乙路、車身業與乙路平行之A車，二車撞擊點應在B車車頭及A車車尾處，然此與本件車損狀態不符，故上訴人所執，僅係個人臆測，不足可採。況且，撞擊點既為A車右前車頭處，是正前方板金無碰撞痕跡自屬當然，而上訴人駕駛A車由南往北駛至案發路口，在左轉欲進入乙路時與由東向西駛至之乙○○所騎機車發生碰撞，當時乙○○所騎B車正沿乙路由東往西方向行駛，依物理慣性定律，B車往前與A車發生撞擊後，所生擠壓力量導致A車右前車頭板金向前凸出，亦與上開物理慣性定律相符，是上訴人所辯，並無可採。從而，上訴人於系爭事故顯有過失，如前所述，其聲請學術單位鑑定及測謊等調查證據，並無必要。㈡被上訴人之損害：⒈本件被上訴人因系爭事故受有系爭傷害，而生醫療費用共計14,939元之損害，業據兩造所不爭執（見原審卷第54頁），故被上訴人此部分之請求，應予准許。⒉又非財產上損害之慰撫金數額，究竟若干為適當，應斟酌兩造身分、地位、經濟狀況、加害程度及其他各種情形，俾為審判之依據（最高法院86年度台上字第511號、第3537號判決意旨參照）。被上訴人因上訴人之不法侵害行為，致受有系爭傷害，除須承受身體上之疼痛，精神上亦受有相當程度之痛苦，及需承受往來就醫之不便，是其依民法第195條第1項前段規定請求慰撫金，自屬有據。本院斟酌被上訴人目前就讀大學，無工作、收入，名下有1輛機車、上訴人係44年出生，專科畢業，從事鞋面加工，及參以兩造之經濟狀況、身分、地位、生活狀況（見原審第55頁、密封袋之稅務電子閘門財產所得調件明細表），並考量上訴人過失之情節及被上訴人所受本案傷害之程度，認被上訴人請求賠償精神慰撫金應以15萬元為適當，逾此範圍不應准許。㈢又按損害之發生或擴大，被害人與有過失者，法院得減輕賠償金額或免除之；前二項之規定，於被害人之代理人或使用人與有過失者，準用之，民法第217條第1項、第3項定有明文。此項規定之目的在謀求加害人與被害人間之公平，倘受害人於事故之發生亦有過失時，由加害人負全部賠償責任，未免失諸過酷，是以賦與法院得不待當事人之主張，減輕其賠償金額或免除之職權。換言之，基於過失相抵之責任減輕或免除，非僅為抗辯之一種，亦可使請求權全部或一部為之消滅，故裁判上得以職權斟酌之（最高法院85年台上字第1756號判決意旨參照）。且法院對於賠償金額減至何程度，抑為完全免除，雖有裁量之自由，但應斟酌雙方原因力之強弱與過失之輕重定之（最高法院97年度台上字第871號判決意旨參照）。又汽車駕駛人，有下列情形之一者，處新臺幣六千元以上一萬二千元以下罰鍰，並當場禁止其駕駛：一、未領有駕駛執照駕駛機車，道路交通安全管理處罰條例第21條第1項第1款亦有明定。未領有駕駛執照而駕駛機車者，係違反道路交通管理處罰條例第21條第1項第1款之規定，自屬違反保護他人之法律，依民法第184條第2項規定，應推定其有過失（最高法院67年台上字第2111號、97年台上字第2095號判決意旨參照）。另行車速度，依速限標誌或標線之規定，無速限標誌或標線者，應依下列規定：…二、行經無號誌之交岔路口，應減速慢行，作隨時停車之準備，道路交通安全規則第93條第1項第2款亦有明訂。查乙○○於警詢時自承：沒有考領適當駕照（見警卷第7頁），係無照駕駛，且於案發地點未減速慢行，作隨時停車之準備，對於系爭事故之發生，顯然亦有過失，前開鑑定意見書與覆議意見書均同此認定，業如前述。本院審酌兩造之過失情節及原因力，認乙○○對系爭事故之發生，亦應負擔35％之肇事責任，始為公允。而乙○○為被上訴人之使用人，故被上訴人自亦應承擔乙○○之過失。依此計算，被上訴人得請求上訴人賠償之金額應酌減為107,210元（計算式：164,939×65％＝107,210.3，小數點以下四捨五入）。㈣按連帶債務人相互間，除法律另有規定或契約另有訂定外，應平均分擔義務；債權人向連帶債務人中之一人免除債務，而無消滅全部債務之意思表示者，除該債務人應分擔之部分外，他債務人仍不免其責任，民法第280條前段、第276條第1項分別定有明文。依此規定，債務人應分擔部分之免除，仍可發生絕對之效力，亦即債權人與連帶債務人中之一人成立和解，如無消滅其他債務人連帶賠償債務之意思，而其同意債權人賠償金額超過「依法應分擔額」（同法第280條）者，因債權人就該連帶債務人應分擔之部分，並無作何免除，對他債務人而言，固僅生相對之效力而無民法第276條第1項之適用，但其應允債權人賠償金額如低於「依法應分擔額」時，該差額部分，即因債權人對其應分擔部分之免除而有民法第276條第1項規定之適用，並對他債務人發生絕對之效力（最高法院98年度台抗字第200號裁定意旨參照）查系爭事故係因上訴人與乙○○之過失所致，故渠等為共同侵權行為，原應依民法第185條第1項規定對被上訴人負連帶之侵權行為損害賠償責任，而內部應分擔額則依兩造過失比例計算，乙○○應負擔損害之35％（計算式：164,939×35％＝57,729元，小數點以下四捨五入，以下均同），惟被上訴人已與乙○○達成以2萬元達成和解並拋棄其餘請求權，有和解書1紙可憑（見原審卷第143頁），該和解書雖免除乙○○對被上訴人所負債務，惟並無一併免除上訴人債務之意，而乙○○之賠償金額低於前開應分擔額，揆諸前述，對於上訴人仍發生清償效力，則依其等內部應分擔額各為35％、65％計算，上訴人仍應對被上訴人負擔65％賠償金額，即107,210元。又保險人依本法規定所為之保險給付，視為被保險人損害賠償金額之一部分，被保險人受賠償請求時，得扣除之，為強制汽車責任保險法第32條所明定。本件被上訴人因系爭事故，已受領強制汽車責任保險金16,724元之事實，為兩造所不爭執，則該保險給付視為上訴人損害賠償金額之一部分，應自得請求上訴人賠償之金額中予以扣除，經扣除前開款項後，被上訴人得請求上訴人給付之金額應為90,486元（計算式：107,210元－16,724＝90,486元）。㈤又按給付無確定期限者，債務人於債權人得請求給付時，經其催告而未為給付，自受催告時起，負遲延責任。其經債權人起訴而送達訴狀，與催告有同一之效力；遲延之債務，以支付金錢為標的者，債權人得請求依法定利率計算之遲延利息；應付利息之債務，其利率未經約定，亦無法律可據者，週年利率為百分之五，民法第229條第2項、第233條第1項前段、第203條分別定有明文。本件被上訴人對上訴人之前揭侵權行為損害賠償請求權，係屬給付無確定期限之金錢債權，而起訴狀繕本係於109年4月30日送達上訴人（見交附民卷第27頁），從而，被上訴人併請求上訴人自109年5月1日起，至清償日止，按法定利率即年息5％計算之利息，自屬有據。二、反訴部分：上訴人固主張被上訴人知悉乙○○為無照飆車駕駛而未制止而有過失，並請求修車費用18,266元及上訴人兩年多往返法院30餘次所生工作損失共計8萬元等情。然按，系爭車禍所生損害係由上訴人及乙○○所造成，如前所述，被上訴人不因前開民法第217條第3項、第1項之適用而對上訴人構成侵權行為；且系爭車禍確實發生，被上訴人於本件主張權利亦非濫訴，故被上訴人應訴係為實現其權利或履行其義務而為，縱因此無法工作，仍無從視為可得請求賠償之權利損害。從而，上訴人反訴之請求均無理由。陸、綜上所述，本件本訴部分，被上訴人依侵權行為損害賠償法律關係，訴請上訴人給付90,486元，及自109年5月1日起至清償日止，按年息5％計算之利息，為有理由，應予准許，逾此部分，則無理由，應予駁回；另反訴部分，上訴人請求被上訴人給付98,266元，則無理由，應予駁回。原審就上開應准許部分中之52,963元為上訴人敗訴之判決（其餘應准許部分，因被上訴人未上訴而確定），並就被上訴人前開勝訴部分依職權宣告假執行，並無不合，上訴意旨指摘原判決不當，求予廢棄改判，為無理由，應駁回其上訴。另原審駁回上訴人之反訴（車損部分），核無違誤；上訴意旨指摘原判決此部分不當，求予廢棄改判，亦為無理由，應駁回其上訴。柒、本件事證已臻明確，兩造其餘攻擊防禦方法及所提證據，核與本件判決結果不生影響，爰不另一一論述，併予敘明。捌、據上論結，本件上訴為無理由，依民事訴訟法第436條之1第3項、第449條第1項、第78條規定，判決如主文。中華民國110年8月11日民事第一庭審判長法官宋國鎮法官劉奕榔法官黃思惠以上正本係照原本作成。本判決不得上訴。書記官林翰章中華民國110年8月11日"

#多人case 暫時不理想，但因為資料量增加，已經比原本進步很多，我覺得資料更多的情況下應該有機會做得很好
#mytext = "臺灣桃園地方法院民事判決107年度訴字第43號原告劉秋芳蔣博宏蔣芯亞蔣志浩共同訴訟代理人蔡尚樺律師複代理人曾雍博律師被告黃建碤訴訟代理人黃程國律師被告謝志陽訴訟代理人黃靖翔謝維陽上列當事人間請求損害賠償事件，本院於民國108年7月24日言詞辯論終結，判決如下：主文一、被告謝志陽應分別給付原告劉秋芳新臺幣182萬5,761元、原告蔣博宏新臺幣8萬元、原告蔣芯亞新臺幣8萬元、原告蔣志浩新臺幣8萬元，及均自民國106年7月15日起至清償日止，按週年利率5%計算之利息。二、原告其餘之訴駁回。三、訴訟費用由被告謝志陽負擔70%，餘由原告負擔。四、本判決第1項於原告劉秋芳、蔣博宏、蔣芯亞、蔣志浩分別以新臺幣60萬9,000元、新臺幣2萬7,000元、新臺幣2萬7,000元、新臺幣2萬7,000元為被告謝志陽供擔保後，得為假執行；但被告謝志陽如分別以新臺幣182萬5,761元、新臺幣8萬元、新臺幣8萬元、新臺幣8萬元為原告劉秋芳、蔣博宏、蔣芯亞、蔣志浩預供擔保，得免為假執行。五、原告其餘假執行之聲請駁回。事實及理由壹、程序方面：按訴狀送達後，原告不得將原訴變更或追加他訴，但擴張或減縮應受判決事項之聲明者，不在此限，民事訴訟法第255條第1項第3款定有明文。本件原告原起訴訴之聲明第1項係請求被告應連帶給付原告劉秋芳（下與原告蔣博宏及蔣芯亞、蔣志浩合稱原告，如單指其一則逕稱其名）新臺幣（下同）266萬1,675元及自起訴狀繕本送達翌日起算之法定遲延利息，嗣於民國108年7月24日具狀變更請求金額為197萬5,426元（見本院卷第311頁），核屬減縮應受判決事項之聲明，應予准許。貳、實體方面：一、原告主張：被告黃建瑛（下與被告謝志陽合稱被告，如單指其一則逕稱其名）於105年5月5日下午3時許，駕駛車牌號碼0000-00號自用小客車（下稱系爭A車）自桃園市○○區○○路○○○○○路○000號路旁欲駛入中華路外側車道時，本應注意前後左右有無車輛並讓行進中之車輛優先通行，且當時並無不能注意之情事，竟疏未注意於此且未顯示方向燈即貿然駛入車道，適劉秋芳騎乘車牌號碼000-000號普通重型機車（下稱系爭B車）自同向後方駛抵，為閃避而往左偏疑，未料謝志陽騎乘車牌號碼000-000號普通重型機車（下稱系爭C車）自同向後方駛來，亦疏未注意車前狀況及減速而自後方追撞，致劉秋芳人車倒地（下稱系爭事故），系爭B車因而毀損，劉秋芳並因此罹有腦震盪症候群、認知功能失調、創傷後人格異常、焦慮、高血壓、睡眠障礙、解離症及腦癲癇症狀（下稱系爭傷勢），迄今生活仍無法自理及工作，而須專人照護，並因系爭事故已支出系爭B車修理費5,850元、醫療費6萬3,800元、就診交通費910元、醫療器材費6,116元及看護費43萬8,000元，另受有勞動能力減損96萬750元及非財產上損害50萬元，共計197萬5,426元。又蔣博宏及蔣芯亞、蔣志浩分別為劉秋芳之配偶及子女，因被告前開侵權行為致劉秋芳受有前開傷勢，亦因此造成其等精神上痛苦不堪，而受有非財產上損害各10萬元，爰依共同侵權行為法律關係提起本件訴訟等語，並聲明：（一）被告應連帶給付劉秋芳197萬5,426元，及自起訴狀繕本送達翌日起至清償日止，按週年利率5%計算之利息；（二）被告應連帶給付蔣博宏、蔣芯亞、蔣志浩各10萬元，及均自起訴狀繕本送達翌日起至清償日止，按週年利率5%計算之利息；（三）願供擔保，請准宣告假執行。二、被告抗辯略以：（一）黃建碤部分：伊駕駛系爭A車自路旁駛入車道之車速相當緩慢，並讓後方車輛通行，而劉秋芳騎乘系爭B車自伊同向後方駛抵前業已查悉伊欲駛入車道而開始煞車減速，於事發前劉秋芳行車速度更接近煞停狀態，堪認伊就維護後方車輛行車安全、防止本件事故發生，已盡相當之注意，而本件經鑑定亦認伊並無肇事因素，至伊未顯示方向燈乙節與系爭事故之發生並無相當因果關係。縱認伊應負侵權責任，然劉秋芳係於事發後5日始至長庚醫療財團法人林口長庚紀念醫院（下稱長庚醫院）就診，並經診斷罹有系爭傷勢，惟系爭事故並非嚴重碰撞，劉秋芳於事發時亦未表明頭部遭撞擊等情，是系爭傷勢是否為系爭事故所致，實屬有疑，原告復未能舉證證明，則因此所生之醫療費、醫療器材費、就診交通費、看護費及勞動能力減損均不得請求，再者，劉秋芳於事發後仍在社群網站上發表文章及直播銷售服飾，情狀與一般常人無異，並無其所稱之失能或意識障礙之情事，且劉秋芳提出之薪資明細並非事發時之薪資，自不足以作為認定損害之基礎等語，資為抗辯，並聲明：（一）原告之訴駁回；（二）如受不利判決，願供擔保請准宣告免為假執行。（二）謝志陽部分：系爭事故係因黃建碤駕駛系爭A車貿然自路旁起駛欲進入車道，劉秋芳為閃避而緊急煞車並車身左偏，致伊未能即時煞車而發生碰撞，伊並無過失。又劉秋芳並無失能之情事，蔣博宏、蔣芯亞、蔣志浩據以請求非財產上損害亦無所據。至就原告請求金額及項目之意見則引用黃建碤之抗辯等語，資為抗辯，並聲明：（一）原告之訴及假執行之聲請均駁回；（二）如受不利判決，願供擔保請准宣告免為假執行。三、兩造不爭執事項（見本院卷第172頁至第173頁，並依判決格式修正或刪減文句，或依爭點論述順序整理內容）：（一）黃建瑛於105年5月5日下午3時許，駕駛系爭A車自中華路145號旁駛出，欲駛入中華路外側車道，適劉秋芳騎乘系爭B車自同向後方駛抵，車身往左偏移因而與同向後方由謝志陽所騎乘系爭C車發生碰撞，致人車倒地。（二）蔣博宏為劉秋芳之配偶，2人並育有蔣芯亞及蔣志浩。（三）被告對於原告提出之原證5至原證9之形式上真正不爭執。（四）系爭B車為訴外人亞博企業社所有，且係101年10月出廠，距系爭事故發生即105年5月5日，已有3年8月車齡。（五）劉秋芳為高中畢業，事發時任職於私立優尼可語文文理補習班（下稱優尼可補習班），蔣博宏為高中肄業，從事水電維修，蔣芯亞及蔣志浩現就學中；黃建瑛為大學畢業，擔任會計人員；謝志陽為大學畢業，擔任工程師。四、原告主張被告應依共同侵權行為規定負連帶損害賠償責任等情，則為被告所否認，並以前詞置辯，是本院應審酌者厥為：（一）被告就系爭事故之發生是否有過失？及被告之行為與劉秋芳所受損害間有無相當因果關係？劉秋芳就系爭事故之發生是否與有過失？（二）原告得請求之損害賠償之金額若干？茲分述如下：(一)關於被告就系爭事故之發生是否有過失而應負共同侵權行為責任之爭點：1、按汽車行駛時，駕駛人應注意車前狀況及兩車並行之間隔，並隨時採取必要之安全措施；起駛前應顯示方向燈，注意前後左右有無障礙或車輛行人，並應讓行進中之車輛行人優先通行，道路交通安全規則第94條第3項、第89條第1項第7款固定有明文，惟按民法第185條第1項前段所謂共同侵權行為，須共同侵權行為人皆已具備侵權行為之要件始能成立，若其中有人無故意過失，或無不法侵害行為，則其人非侵權行為人，自不須與具備侵權行為要件之人負連帶損害賠償責任。且按侵權行為之成立，須行為人因故意過失不法侵害他人權利，亦即行為人須具備歸責性、違法性，並不法行為與損害間有因果關係，始能成立。（最高法院100年度台上字第328號判決意旨參照）。2、依據卷附道路交通事故現場圖及照片所示（見本院卷第83頁、第92頁至第93頁），肇事地點位於中華路145號前之外側車道，黃建瑛係駕駛系爭A車自路旁欲駛入外側車道，劉秋芳則係騎乘系爭B車由南向北（即中壢往桃園方向）駛來，謝志陽亦騎乘系爭C車自同向後方駛來，其中系爭A車車頭朝西北停於路旁及機車道上，系爭B車車頭亦朝西北停於外側車道及機車道上，而經本院當庭勘驗謝志陽於事發時行車紀錄器畫面，結果為：「一、畫面開啟時，畫面右下角時間為『2016/05/1514：59:42』，謝志陽騎乘機車於中華路與龍壽街交岔路口停等紅燈。二、畫面開啟24秒，畫面右下角時間為『2016/05/1515：00:07』，紅綠燈變為綠燈，謝志陽起駛直行中華路。三、畫面開啟1分18秒，畫面右下角時間為『2016/05/1515：01：00』，謝志陽前方可見劉秋芳（身著白色條紋上衣）騎乘機車直行於機車道，另黃建碤駕駛白色小客車自路邊緩慢駛出欲往左側駛入慢車道，左前輪部分已駛入機車道內。四、畫面開啟1分19秒，畫面右下角時間為『2016/05/1515：01：01』，劉秋芳機車與黃建碤所駕駛汽車約有2輛汽車距離，此時劉秋芳機車之煞車燈亮起，謝志陽則騎乘機車行駛於後方慢車道上。五、畫面開啟1分22秒，畫面右下角時間為『2016/05/1515：01：05』，劉秋芳騎乘機車直行於機車道上，煞車燈仍係亮起狀態，黃建碤駕駛汽車自路邊往左緩慢行駛，左側車身已進入機車道。六、畫面開啟1分23秒，畫面右下角時間為『2016/05/1515：01：06』，劉秋芳騎乘之機車由機車道往左偏駛慢車道，煞車燈仍係亮起狀態，黃建碤仍駕駛汽車往左駛，謝志陽則騎乘機車行駛於劉秋芳後方慢車道上約2輛汽車距離。七、畫面開啟1分24秒，畫面右下角時間為『2016/05/1515：01：07』，劉秋芳機車此時趨近停止狀態，煞車燈仍係亮起狀態，與黃建碤汽車車尾約半輛汽車距離，謝志陽則騎乘機車自左後方撞擊劉秋芳機車。八、畫面開啟1分25秒，畫面右下角時間為『2016/05/1515：01：08』，謝志陽機車自左側倒下後滑行。九、畫面開啟1分28秒，畫面右下角時間為『2016/05/1515：01：11』謝志陽機車停止不動。」有本院勘驗筆錄在卷可佐（見本院卷第308頁至第309頁），可見黃建瑛在路旁起駛欲進入外側車道前其車速極慢，並無驟然加速駛入車道之情，且依劉秋芳之行駛狀態，劉秋芳於畫面開啟1分19秒時已開始煞車減速以避免危險，直至畫面開啟1分24秒即碰撞發生前已趨近停止，並非無預警驟停或偏移、變換車道，謝志陽自後方駛抵如有注意車前狀況即得查悉有追撞之危險，並得選擇減速或其他避讓措施以避免系爭事故之發生，卻未為之，堪認系爭事故之發生係因謝志陽未注意車前狀況，未與前車保持隨時得煞停之距離，以致追撞劉秋芳機車，謝志陽之駕駛行為顯有過失無疑，至黃建瑛就維護後方車輛行車安全、防止本件事故發生，已盡相當之注意，尚難認黃建瑛有違反道路交通安全規則之過失，且系爭事故經送請桃園市政府交通局車輛行車事故鑑定覆議委員會覆議結果亦同此認定，有該會桃市覆0000000號鑑定覆議意見書在卷可參（見本院卷第282頁至第285頁），益證謝志陽就系爭事故之發生有過失，黃建瑛則無過失。3、原告雖主張黃建瑛起駛前未顯示方向燈，已違反道路交通安全規則第89條第1項第7款規定，屬違反保護他人法律云云，然如前述，黃建瑛於起駛進入外側車道前車速緩慢，並有讓後方直行車之通行，又劉秋芳騎乘系爭B車於事發前數秒鐘已有數度煞車減速之情事，應係已發現黃建瑛欲自路旁駛入車道一事，且系爭事故之發生係因謝志陽未注意車前狀況而追撞劉秋芳所致，縱黃建瑛未顯示方向燈亦與系爭事故之發生無相當因果關係，亦無從據以即認黃建瑛應負侵權行為責任。4、至本件肇事原因前經桃園市政府車輛行車事故鑑定會鑑定認：「一、謝志陽駕駛普通重型機車行經中央劃分島路段，未充分注意車前狀況，自後追撞前車，為肇事主因。二、黃建碤駕駛自小客車行經中央劃分島路段，自路旁起駛未開啟方向燈光且未讓車道上行經中之車輛先行，為肇事次因。三、劉秋芳駕駛普通重型機車無肇事因素。」有該會桃市鑑0000000案鑑定意見書附卷可參（見本院卷第202頁至第203頁反面），惟上開鑑定意書中之「未開啟方向燈」與系爭事故之發生並無相當因果關係，且黃建瑛起駛前已禮讓後方直行車通行，並無過失，業如前述，從而，上開鑑定意見，除與覆議意見不符外，又未能充分審酌本件事證，其逕認黃建瑛之駕車行為亦為系爭事故之肇事原因云云，不僅與卷證有所出入，亦與本院前揭認定之事實不同，尚不足為原告主張之有利證明，自不足採信。5、被告雖抗辯劉秋芳對於系爭事故之發生亦與有過失云云，惟如前述，劉秋芳係無預警遭謝志陽自後方追撞，而劉秋芳騎乘系爭B車於同向前方車道行駛，本能信賴其他車輛亦遵守交通規則，且謝志陽此際並非毫無防止危害發生之能力，實難認劉秋芳對於謝志陽此舉有何預見可能性，遑論可期待劉秋芳對此猝不及防之事加以注意，自無從將系爭事故發生之原因歸責於劉秋芳之駕駛行為，且經本院送請桃園市政府交通局車輛行車事故鑑定覆議委員會鑑定亦同此認定，已如前述，益徵劉秋芳就系爭事故之發生並無過失，此外，被告復未能就劉秋芳對系爭事故之發生與有過失一事舉證以實其說，自非可採，是被告此部分所辯，自屬無據。6、又被告質疑系爭事故碰撞並非嚴重，且係距事發後數日始至長庚醫院就診，是劉秋芳所罹之系爭傷勢應與系爭事故無關云云，而查，觀諸衛生福利部桃園醫院（下稱桃園醫院）之病歷（見本院卷第186頁至第191頁），劉秋芳於事發後至桃園醫院急診，並經診斷為「頭皮鈍傷」、「腦震盪」，另於105年5月23日再次就診，經診斷為「癲癇」、「非特定的焦慮症」等情，另本院檢附前開病歷資料函請長庚醫院說明原告所罹系爭傷勢之成因為何？與系爭事故有無關連性？經該院以108年3月12日長庚院林字第1080150075號函覆表示：「「二、據病歷所載，病人劉女士（本院按即劉秋芳）自105年5月20日起陸續至本院腦功能暨癲癇科就醫之診斷為車禍發生的行為異常及焦慮、憂鬱情緒，並接受藥物及安排精神科回診治療，而依病人上開就醫期間之病情研判，病人自述於車禍發生後發生焦慮、憂鬱情緒，惟因車禍部分並無至本院急診就醫之紀錄，故本院醫師無法判斷病人上開病症與其自述車禍間之關聯性……三、據病歷所載，病人劉女士自105年6月11日起陸續至本院精神科就醫之診斷為慢性創傷後壓力疾患、伴有抽搐或痙攣之轉化症、腦震盪後症候群、腦波檢查結果異常、其他憂鬱發作，並接受藥物及支持性心理治療，而依病人上開就醫期間之病情研判，因病人有腦震盪與事故相關之心理壓力，故本院醫師研判其上開病症與自述車禍間應有其關聯性……」（見本院卷第278頁及反面），足見劉秋芳因系爭事故受有頭皮鈍傷及腦震盪之傷害，嗣又併發認知功能失調、創傷後人格異常、焦慮及腦癲癇等症狀，堪認系爭傷勢確係肇因於系爭事故所致，而有相當因果關係甚明，被告前開所辯，自無足採。準此，系爭事故之發生係因謝志陽之過失行為所致，而與黃建瑛無涉，已如前述，則劉秋芳依侵權行為法律關係及民法第185條規定，請求被告負連帶賠償責任，即屬無據，本件應由謝志陽負侵權行為之損害賠償責任，堪予認定。（二）關於原告得請求之損害賠償金額之爭點：1、按不法侵害他人身體或健康者，對於被害人因此喪失或減少勞動能力或增加生活上之需要時，應負損害賠償責任；又不法侵害他人之身體、健康、名譽、自由、信用、隱私、貞操，或不法侵害其他人格法益而情節重大者，被害人雖非財產上之損害，亦得請求賠償相當之金額，民法第193條第1項、第195條第1項前段分別定有明文。本件劉秋芳因謝志陽之過失，致受有系爭傷勢，已如前述，揆諸首揭規定，劉秋芳自得請求謝志陽負損害賠償責任。茲就劉秋芳請求之各項賠償金額，審究如下：（1）醫療費及醫療器材費部分：查劉秋芳所受系爭傷勢與系爭事故有相當因果關係，已如前述，而劉秋芳主張因系爭傷勢已支出醫療費6萬3,800元及醫療器材費6,116元等情，業據其提出與其所述相符之醫療費用收據及統一發票為證（見本院卷第24頁至第61頁），核屬因系爭事故發生而增加之必要支出，是劉秋芳此部分之請求，應予准許。（2）就診交通費部分：劉秋芳主張因系爭事故須就醫而搭程計程車往返醫院而支出交通費910元乙節，業據其提出計程車乘車證明為憑（見本院卷第54頁），而比對劉秋芳提出之醫療費用收據之就診日期，僅106年5月6日確有至長庚醫院就診（見本院卷第53頁），則劉秋芳於該日支出之計程車費用295元堪認因系爭事故所增加生活上之需要，自應准許。至其餘交通費部分尚無證據證明係因至醫療院所就醫所支出，自難准許。（3）看護費部分：劉秋芳主張其因系爭事故受有系爭傷勢，迄今生活仍無法自理，自事發起至1年均由家人照護，而受有看護費43萬8,000元之損害等情，已據其提出長庚醫院105年5月31日、105年10月8日診斷證明書為憑（見本院卷第19頁、第20頁），而經本院函詢長庚醫院，經該院函覆：「……病人自受傷時起應無法自理生活而有專人照護之需要，本院醫師研判病人至2017年4月8日回診時始能自理生活。……」，有長庚醫院前開函文在卷可參，堪認劉秋芳主張其自系爭事故發生（105年5月15日）起至106年4月8日止均有僱請看護全日照護之必要，至超過上開期間部分，劉秋芳並未能證明仍有看護之必要，自難准許。又親屬看護時雖無現實看護費之支付，仍應認被害人受有相當於看護費之損害，並得向加害人請求，始符公平原則（最高法院89年度台上字第1749號、92年度台上字第431號、94年度台上字第1543號判決意旨參照），而謝志陽對劉秋芳主張看護費每日以1,200元計算乙節並不爭執（見本院卷第159頁），依此計算之看護費為39萬4,800元（計算式：1,200元×329日＝39萬4,800元），應予准許。（4）系爭B車修理費部分：劉秋芳雖主張謝志陽因駕車不慎損害系爭B車，請求賠償修理費5,850元等情，然查，系爭B車並非劉秋芳所有，而係亞博企業社所有，為劉秋芳所是認【見兩造不爭執事項（四）】，是劉秋芳既非所有權人，縱認系爭B車確因系爭事故受有車體損害等情為真，仍不可認為係劉秋芳之權利受到損害，而經本院闡明（見本院卷第158頁反面)後，仍僅空言稱平日係由劉秋芳使用等語，復未提出已自亞博企業社受讓系爭事故侵權行為損害賠償債權之證明，故劉秋芳以其本人名義向謝志陽請求賠償系爭B車之修理費，即難認有據，洵屬不能准許。（5）勞動能力減損部分：①按被害人因身體健康被侵害而喪失勞動能力所受之損害，其金額應就被害人受侵害前之身體健康狀態、教育程度、專門技能、社會經驗等方面酌定之，不能以一時一地之工作收入為準，應以其能力在通常情形下可能取得之收入為標準（最高法院61年台上字第1987號、63年台上字第1394號判例參照）。又依民法第193條第1項命加害人一次支付賠償總額，以填補被害人所受喪失或減少勞動能力之損害，應先認定被害人因喪失或減少勞動能力而不能陸續取得之金額，按其日後本可陸續取得之時期，各照霍夫曼式計算法，扣除依法定利率計算之中間利息，再以各時期之總數為加害人一次所應支付之賠償總額，始為允當（最高法院22年上字第353號判例參照）。②經查，原告於事發時原任職於優尼可補習班工作【見兩造不爭執事項（五）】，核其工作性質除偏向體力活動外尚須常借助思考或智力方可完成其工作內容，再參以長庚醫院108年5月20日長庚院林字第1080350362號函所述：「……據病歷所載，病人劉君於108年4月24日至本院職業醫學科門診接受勞動力減損評估，經專科醫師依病人現況進行問診、理學檢查（含簡易智能狀態測驗，MMSE）正常、病歷審閱，且據本院105年5月31日出院病歷摘要記載，病人曾至署立桃園醫院接受腦部電腦斷層檢查結果並無異常發現等相關評估結果顯示，病人劉君因頭部外傷併發性格改變，致現狀遺存解離性人格異常、焦慮等症；根據美國醫學會障害指引評估指南（第六版）及美國加州永久失能評估準則之評核標準，加以綜合病人劉君將來賺錢能力、工作性質及年齡予以調整計算鑑定後（計算公式如附件），其勞動力減損21%。」（見本院卷第286頁至第287頁），認劉秋芳勞動能力減損以21%為準，應屬妥適，是被告辯稱劉秋芳並無減少勞動能力云云，並非可採。③復按勞工未滿65歲者，雇主不得強制其退休，勞動基準法第54條第1項第1款亦有明定。查劉秋芳係60年7月11日出生（見本院卷第19頁），於105年5月15日發生系爭事故算至125年7月10日年滿65歲，原告尚可工作20年1月26日，本院審酌劉秋芳於事發時每月薪資為2萬8,000元，及前述劉秋芳所受減損勞動能力之比例21%計算，劉秋芳每年減損勞動能力之損害額應為7萬560元（計算式：2萬8,000元×12月×21%＝7萬560元），而依霍夫曼式計算法扣除中間利息（首期給付不扣除中間利息），核計其金額為100萬1,483元【計算式：7萬560元×14.00000000+(7萬560元×0.00000000)×(14.00000000-00.00000000)=100萬1,482元，元以下四捨五入。其中14.00000000為年別單利5%第20年霍夫曼累計係數，14.00000000為年別單利5%第21年霍夫曼累計係數，0.00000000為未滿一年部分折算年數之比例(1/12+26/365=0.00000000)】，而劉秋芳僅請求96萬750元，洵屬有據。（6）精神慰撫金部分：按精神慰撫金之賠償，核給之標準固與財產上損害之計算不同，然非不可斟酌雙方身分資力與加害程度，及其他各種情形核定相當之數額（最高法院51年台上字第223號判例意旨參照）。是精神慰撫金之多寡，應斟酌雙方之身分、地位、資力、加害之程度、被害人所受之痛苦及其他各種情形，核定相當之數額。玆審酌劉秋芳及謝志陽之學經歷狀況【見兩造不爭執事項（五）】暨兩造財產所得調件明細表（見本院卷第138頁至第139頁、第149頁至第151頁，僅供本院斟酌精神慰撫金數額之用，不予在判決中詳細列載公開）、身分、年齡、地位、經濟狀況、謝志陽加害之程度及劉秋芳所受損害程度等一切情狀，認劉秋芳得請求之精神慰撫金以40萬元為適當，逾此範圍之請求，不應准許。（7）承上，劉秋芳所得請求被告賠償之損害合計182萬5,761元（計算式：醫療費6萬3,800元＋醫療器材費6,116元＋看護費39萬4,800元＋就診交通費295元＋勞動能力減損96萬750元＋精神慰撫金40萬元＝182萬5,761元）。2、又按不法侵害他人基於子女關係之身分法益而情節重大者，被害人雖非財產上之損害，亦得請求相當之金額，民法第195條第1項、第3項定有明文。查蔣博宏為劉秋芳之配偶，蔣芯亞及蔣志浩為劉秋芳之子女，劉秋芳因系爭事故受有系爭傷勢，日常生活亟需他人協助、照護，其等基於與劉秋芳之父母子女及配偶關係之親情、倫理及生活相互扶持與幫助之身分法益已受到侵害，核其情節自屬重大，是蔣博宏、蔣芯亞及蔣志浩依前揭規定請求謝志陽賠償非財產上損害，即屬有據。玆審酌蔣博宏、蔣芯亞及蔣志浩及謝志陽之學經歷狀況【見兩造不爭執事項（五）】暨兩造財產所得調件明細表（見本院卷第140頁至第151頁，僅供本院斟酌精神慰撫金數額之用，不予在判決中詳細列載公開）、身分、年齡、地位、經濟狀況、謝志陽加害之程度及蔣博宏、蔣芯亞及蔣志浩所受損害程度等一切情狀，認其等各得請求之精神慰撫金以8萬元為適當，逾此範圍之請求，不應准許。五、綜上所述，原告依侵權行為法律關係，請求謝志陽分別給付劉秋芳182萬5,761元、蔣博宏8萬元、蔣芯亞8萬元、蔣志浩8萬元，及均自起訴狀繕本送達翌日起（即106年7月15日，見本院卷第117頁）至清償日止，按週年利率5%計算之利息，為有理由，應予准許；至逾此範圍之請求，尚屬無據，應予駁回。六、假執行之宣告：本件原告勝訴部分，兩造均陳明願供擔保，請准宣告假執行或免為假執行，經核於法並無不合，爰分別酌定相當之擔保金額宣告之；至原告敗訴部分，其假執行之聲請，因訴之駁回而失所依據，不予准許。七、本件事證已臻明確，兩造其餘攻擊防禦方法及舉證，核與判決結果無影響，爰不予一一論述，併此敘明。八、訴訟費用負擔之依據：民事訴訟法第79條、第85條第2項。中華民國108年8月23日民事第二庭法官張永輝正本係照原本作成。如對本判決上訴，須於判決送達後20日內向本院提出上訴狀。如委任律師提起上訴者，應一併繳納上訴審裁判費。中華民國108年8月23日書記官劉雅婷"

#=======================

#Correct
#mytext = "臺灣臺南地方法院臺南簡易庭民事判決109年度南簡字第1469號原告楊健成被告楊制穎上列被告因過失傷害案件，原告提起刑事附帶民事訴訟，請求侵權行為損害賠償，經本院刑事庭裁定（109年度交簡附民字第123號）移送前來，本院於民國109年11月26日言詞辯論終結，判決如下：主文被告應給付原告新臺幣6,524元，及自民國109年8月20日起至清償日止，按週年利率百分之5計算之利息。原告其餘之訴駁回。訴訟費用由被告負擔3分之1，餘由原告負擔。本判決原告勝訴部分得假執行。事實及理由一、被告經合法通知，未於言詞辯論期日到場，核無民事訴訟法第386條所列各款情形，爰依原告之聲請，由其一造辯論而為判決。二、原告主張：被告於民國109年3月4日23時45分許違規駕駛車牌號碼000-0000號自用小客車（下稱系爭汽車）撞傷騎乘車牌號碼000-000普通重型機車（下稱系爭機車）之被告，此過失傷害案件業經臺灣臺南地方檢察署（下稱臺南地檢署）109年度偵字第7484號聲請簡易判決處刑，爰提起本訴，請求被告賠償原告醫藥費新臺幣（下同）1萬元、精神賠償金14萬元。並聲明：被告應給付原告15萬元，及自起訴狀繕本送達翌日起至清償日止，按週年利率百分之5計算之利息。三、被告未於言詞辯論期日到場，亦未提出書狀作何聲明或陳述。四、得心證之理由：(一)經查，被告於109年3月4日23時45分許駕駛系爭汽車，在劃設紅實線、禁止臨時停車之臺南市○○區○○路0段000號前，佔用該址前方之府前路1段西向東外側車道及禁止臨時停車之路緣停放車輛；適原告騎乘系爭機車沿府前路1段由西向東行至該處，機車手把擦撞系爭汽車，原告因而人車倒地，並受有骨盆挫傷、大腿挫傷、手部挫傷、小腿挫傷、小腿擦傷等傷害。又被告上開行為，業經臺南地檢署以109年度偵字第7484號聲請簡易判決處刑，經本院以109年度交簡字第2154號刑事判決（下稱刑案）認被告因過失傷害人，處拘役10日，如易科罰金，以1,000元折算1日等事實，業據本院調取上開刑案卷宗，並核閱其內所附交通事故談話紀錄表、調查筆錄、原告之臺南市立醫院診斷證明書、道路交通事故現場圖、道路交通事故調查報告表㈠、㈡、現場照片、監視器畫面截圖等件查明無訛（見刑案警卷第3至9、14至16、25至37頁）。被告於刑案警詢時，亦對其於上開時、地，有在劃設紅線路段熄火停車之行為坦承不諱（見刑案警卷第3頁），是上開事實均堪信為真實。(二)按駕駛人駕駛汽車，應遵守道路交通標誌、標線、號誌之指示。紅實線設於路側，用以禁止臨時停車。汽車臨時停車時，應依下列規定：三、設有禁止臨時停車標誌、標線處所不得臨時停車。汽車停車時，應依下列規定：一、禁止臨時停車處所不得停車。九、顯有妨礙其他人、車通行處所，不得停車。道路交通標誌標線號誌設置規則第149條第1項第1款第5目、道路交通安全規則第90條前段、第111條第1項第3款、第112條第1項第1、9款分別定有明文。被告將系爭汽車停放在禁止停車之處所，且已佔據部分道路路面，顯足妨礙其他人、車通行，此有上開現場照片及道路交通事故現場圖附卷可稽，而本件事發當時天候晴、夜間有照明、柏油路面乾燥無缺陷、無障礙物、視距良好，有上開道路交通事故調查報告表㈠可按，並無不能注意之情事，詎被告竟疏未注意上情，在禁止停車且足妨礙其他人、車通行之處所停車，而釀致本件車禍，有注意義務之違反甚明。另本件交通事故經臺南市車輛行車事故鑑定委員會鑑定結果，亦認被告駕駛自小客車，於禁止臨時停車處，佔用車道停車，妨礙交通，為肇事次因，有鑑定意見書附卷可參（見刑案偵卷第10頁正、反面），亦與本院上開認定相符。而原告因本件交通事故，受有骨盆挫傷、大腿挫傷、手部挫傷、小腿挫傷、小腿擦傷等傷害，是被告上開過失行為與原告所受傷害，有相當因果關係，亦甚明確。(三)次按因故意或過失，不法侵害他人之權利者，負損害賠償責任。不法侵害他人之身體、健康、名譽、自由、信用、隱私、貞操，或不法侵害其他人格法益而情節重大者，被害人雖非財產上之損害，亦得請求賠償相當之金額。民法第184條第1項前段、第195條第1項前段分別定有明文。被告既因上開過失行為致原告受有傷害，自應對原告所受之損害，負損害賠償責任。爰就原告請求被告賠償之項目及金額，准許與否，審酌如下：1.醫藥費：原告主張因本件車禍所受傷害，支出醫療費用乙情，業據其提出門診收據為證（本院卷第61、62頁），惟上開醫療收據所載費用合計僅為1,310元，且別無證據證明原告另有支出其他醫療費用，是原告得請求賠償之醫療費即為1,310元，逾此部分之請求，則屬無據。2.精神慰撫金：按被害人受有非財產上損害，請求加害人賠償相當金額之慰撫金時，法院對於慰撫金之量定，應斟酌實際加害情形、所造成之影響、被害人痛苦程度、兩造之身分、地位、經濟情形及其他各種狀況，以核定相當之數額。原告因本件交通事故受有上開傷害，造成其日常生活起居之不便且須至醫院治療，其精神上自受有相當之痛苦，原告據此請求精神慰撫金，應屬有據。經審酌原告為49年生、國小肄業，於107年度收入總額261,100元、108年度則無收入，名下無財產；被告為83年生、大學肄業，於107年度收入總額21,891元、108年度無收入，名下有不動產7筆，財產總額為770,433元，有兩造稅務電子閘門財產所得調件明細表、臺南市政府警察局第二分局調查筆錄、個人戶籍資料查詢結果在卷可稽（本院卷第27至37頁、刑案警卷第4、7、10、12頁），併衡酌原告之傷勢輕重程度、被告違反注意義務之情節等一切情狀，認原告請求被告給付之精神損害賠償金額於15,000元之範圍內，應為適當，逾此部分之請求為無理由，不應准許。3.以上，原告所受損害合計為16,310元（計算式：醫藥費1,310元＋精神慰撫金15,000元＝16,310元）。(四)再按損害之發生或擴大，被害人與有過失者，法院得減輕賠償金額，或免除之，民法第217條第1項定有明文。又汽車行駛時，駕駛人應注意車前狀況，並隨時採取必要之安全措施。道路交通安全規則第94條第3項定有明文。本件事發路段筆直寬闊，有上開現場照片可參，原告騎乘系爭機車行駛至上開路段，應可注意到系爭汽車停放於其行向前方之路旁，而本件事故發生當時，並無不能注意之情事，已如前述，原告疏於注意車前狀況，致與系爭汽車發生擦撞，同有注意義務之違反。又本件事故經臺南市車輛行車事故鑑定委員會鑑定，亦認原告騎乘系爭機車，未注意車前狀況，為肇事主因，足見原告就本件車禍之發生與有過失。本院審酌兩造違反注意義務之情節、態樣，認原告就本件事故發生所受損害應負擔百分之60過失責任，而由被告負擔百分之40之過失責任，故被告之賠償責任以減輕百分之60為當。是經過失相抵結果，原告得向被告請求之數額應為6,524元【16,310元×（1－60%）＝6,524元】。(五)末按給付無確定期限者，債務人於債權人得請求給付時，經其催告而未為給付，自受催告時起，負遲延責任；其經債權人起訴而送達訴狀，或依督促程序送達支付命令，或為其他相類之行為，與催告有同一之效力；又遲延之債務以支付金錢為標的者，債權人得請求依法定利率計算之遲延利息；而應付利息之債務，其利率未經約定，亦無法律可據者，週年利率為百分之5，民法第229條第2項、第233條第1項及第203條分別定有明文。查原告請求被告給付之前開金額，未定有給付之期限，則原告併請求自附帶民事起訴狀繕本送達被告之翌日即109年8月20日（見本院附民卷第9頁之送達證書）起至清償日止，按週年利率百分之5計算之法定遲延利息，亦屬有據。五、綜上所述，原告依侵權行為之法律關係，請求被告給付6,524元，及自109年8月20日起至清償日止，按週年利率百分之5計算之利息，為有理由，應予准許。逾此範圍之請求，則無理由，應予駁回。六、本件係屬民事訴訟法第427條第1項簡易訴訟事件，所為被告部分敗訴之判決，爰依同法第389條第1項第3款之規定，就原告勝訴部分依職權宣告假執行。七、按各當事人一部勝訴、一部敗訴者，其訴訟費用，由法院酌量情形，命兩造以比例分擔或命一造負擔，或命兩造各自負擔其支出之訴訟費用。民事訴訟法第79條定有明文。本院審酌原告之請求內容、訟爭之起因及兩造勝敗比例等情，爰確定本件訴訟費用之負擔如主文第三項所示。八、據上論結，本件原告之訴為一部有理由、一部無理由，依民事訴訟法第436條第2項、第385條第1項前段、第79條、第389條第1項第3款，判決如主文。中華民國109年12月10日臺南簡易庭法官張玉萱以上正本係照原本作成。如不服本判決，應於送達後20日內，向本院提出上訴狀並表明上訴理由，如於本判決宣示後送達前提起上訴者，應於判決送達後20日內補提上訴理由書（須附繕本），如委任律師提起上訴者，應一併繳納上訴審裁判費。中華民國109年12月10日書記官鄭伊汝"
#mytext = "臺灣新北地方法院三重簡易庭民事判決111年度重簡字第785號原告方嘉祥被告陳茂順訴訟代理人呂樹緯上列當事人間請求侵權行為損害賠償事件，於民國111年11月11日言詞辯論終結，本院判決如下：主文被告應給付原告新臺幣玖拾貳萬伍仟陸佰玖拾壹元，及自民國一百一十一年三月二十九日起至清償日止，按年息百分之五計算之利息。原告其餘之訴及其餘假執行之聲請均駁回。訴訟費用由被告負擔百分之二十，餘由原告負擔。本判決原告勝訴部分得假執行。事實及理由一、原告起訴主張：被告於民國109年11月4日9時41分許，駕駛車牌號碼000-0000號自用小客車，沿新北市三重區疏洪十二路往疏洪五路2段方向行駛，行經疏洪十二路中山棒球場旁，本應注意行經劃有分向限制線之路段，不得駛入來車之車道內，而依當時情形，又無不能注意之情事，竟疏未注意，貿然跨越分向限制線駛入來車道，適原告騎乘所有車牌號碼000-0000號普通重型機車（下稱系爭機車）沿疏洪十二路往疏洪一路方向駛至，兩車發生碰撞，造成原告受有左側脛骨幹併腓骨幹粉碎開放性骨折、左側近端腓骨骨折併脫位及左側腓神經受損等傷害，系爭機車亦毀損，原告因此受有下列損害共新臺幣（下同）5,090,790元，應由被告負損害賠償責任：①系爭機車受損後經送修而支出修復費用43,450元（均為材料費）；②受傷就醫因而支出醫療費用113,576元；③因傷需專人看護，受有支出看護費用48萬元之損害；④因傷不良於行，搭乘計程車至醫院就診，因而支出交通費用15,831元；⑤原告擔任電機工程師，主要負責網路監視設備、出入識別等工作，月薪34,000元（每日工資1,134元），因傷需休養16月，受有工作損失544,000元（計算式：34,000元×16月＝544,000元）；⑥原告受傷後左腳整個扭曲變形，導致粉碎性骨折以及開放性骨折，還看到自己的腳踝整個反轉，痛到完全不行，期間經過了三次手術才變成内固定，但原告腿骨粉碎了10公分，到現在拍X光都還是三節，住院期間挨了無數的針，沒有止痛針的話，根本無法承受，直至現在仍需吃止痛藥，且原告有家庭要養，小孩也還小，然只能依靠妻子跟原本儲蓄來負擔家計，深感自己是個廢物，目前車禍至今已1年多，骨頭仍是未長全的狀態，無法正常走路，期間多位醫生看過X光片均推斷說至少需要一年半以上才會長好，更須長期復健，原告實承受莫大之精神痛苦，故請求被告賠償非財產上之損害即慰撫金380萬元）；⑦原告受傷後，經醫院預估復健時間至少要半年，須支出復健費用93,933元。為此，爰依侵權行為之法律關係，提起本件訴訟，並聲明請求：被告應給付原告5,090,790元，及自起訴狀繕送達翌日起至清償日止，按年息5%計算之法定遲延利息等事實。二、被告則求為判決駁回原告之訴，並辯稱：對事故之肇事責任及原告支出之醫療費用、交通費用（有單據者）並不爭執，惟系爭機車之修復材料費應予折舊；至於原告之其他請求，請法院查明認定等情。三、原告主張於前揭時、地遭被告不法過失傷害身體及毀損系爭機車等事實，業據其提出馬偕紀念醫院診斷證明書、醫療費用收據、新北市政府車輛行車事故鑑定會新北車鑑字第0000000號鑑定意見書、機車修車估價單等為證，復為被告所不爭執，堪認被告已構成過失不法侵害原告身體、系爭機車所有權之侵權行為甚明。四、按因故意或過失，不法侵害他人之權利者，負損害賠償責任；又不法侵害他人之身體或健康者，對於被害人因此喪失或減少勞動能力或增加生活上之需要時，應負損害賠償責任；另不法侵害他人之身體、健康、名譽、自由、信用、隱私、貞操，或不法侵害其他人格法益而情節重大者，被害人雖非財產上之損害，亦得請求賠償相當之金額，民法第184條第1項前段、第193條第1項、第195條第1項前段分別定有明文。本件被告既因過失行為致原告身體受傷及系爭機車亦受損，已如前述，則原告主張被告應負損害賠償責任，自屬有據。茲就原告得請求被告賠償之項目及金額分別審酌如下：（一）系爭機車修復費用部分：按物被毀損時，被害人除得依民法第196條請求賠償外，並不排除民法第213條至第215條之適用。而依民法第196條請求賠償物被毀損所減少之價額，得以修復費用為估定之標準，但以必要者為限（例如︰修理材料以新品換舊品，應予折舊）〔最高法院77年度第9次民事庭會議決議(1)意旨參照〕。本件系爭機車之修復係以新零件更換受損之舊零件，則以修復費作為損害賠償之依據時，自應將零件折舊部分予以扣除。查系爭機車係於105年10月出廠使用，有車號查詢機車車籍表在卷可佐，至109年11月4日受損時，已使用逾3年，而本件修復費用為43,450元（均為材料費），有修車估價單在卷可憑。本院依依行政院所頒「固定資產耐用年數表」及「固定資產折舊率表」之規定，可知機械腳踏車之耐用年數為3年，依定率遞減法每年折舊千分之536，其最後一年之折舊額，加歷年折舊累積額，總和不得超過該資產成本原額之10分之9之計算結果，系爭機車之折舊年數既已逾3年，則其修復材料費扣除折舊後之餘額為10分之1即4,345元，此即為是原告得請求被告賠償之修復費用。（二）醫療費用部分：業據原告提出馬偕紀念醫院診斷證明書、醫療費用收據等為證，並為被告所不爭執，則原告請求被告賠償醫療費用113,576元，即屬有據。（三）看護費用部分：本件原告雖主張因傷需專人看護，受有支出看護費用48萬元之損害等情。然經本院依職權就原告所受傷勢，於受傷期間是否須由他人專門看護日常生活起居？如是，看護期間須多久等事項，向馬偕紀念醫院查詢，結果覆稱：「....方君因左側脛骨粉碎、開放性骨折於109年11月4日至本院急診求治，接受清創及固定手術，並於109年11月18日出院。一般而言，骨折手術....；三個月期間活動不便，需他人看護。」此有該院111年7月18日馬院醫骨字第1110004625號函附卷可稽。足見原告所受傷勢應專人看護期間為3個月。另本院參諸全日看護費用按照醫療院所收費標準，平均約為2,200元，依此核算，原告得請求被告賠償之看護費用應為198,000元（計算式：2,200元×30天×3月＝198,000元）。（四）交通費用部分：雖據原告提出計程車乘車證明等為證，惟經核前揭單據之金額僅為15,090元，原告逾此範圍之請求，非屬有據。（五）工作損失部分：本件原告固主張擔任電機工程師，主要負責網路監視設備、出入識別等工作，月薪34,000元（每日工資1,134元），因傷需休養16月，受有工作損失544,000元等情。然經本院依職權就原告所受傷勢，自受傷日起須休養多久始能再從事一般網路監視系統工程師之工作之事項，向馬偕紀念醫院查詢，結果覆稱：「....方君因左側脛骨粉碎、開放性骨折於109年11月4日至本院急診求治，接受清創及固定手術....。一般而言，骨折手術三個月後可從事一般工作」，此有該院前開函文附卷可稽，自應認原告之傷勢應休養之期間為3個月；又關於原告之月薪資，依原告所提出之109年10月份薪資單核算，其月薪應為「30,060元」（即薪俸22,533元＋經常性給與7,527元），並非原告主張之34,000元。據此核算，原告所受工作損失金額應為90,180元（計算式：30,060元×3月＝90,180元）。（六）慰撫金部分：按慰藉金之賠償須以人格權遭遇侵害，使精神上受有痛苦為必要，其核給之標準固與財產上損害之計算不同，然非不可斟酌雙方身分資力與加害程度，及其他各種情形核定相當之數額（最高法院51年台上字第223號判例要旨參照，但本則判例，依據108年1月4日修正，108年7月4日施行之法院組織法第57條之1第2項，其效力與未經選編為判例之最高法院裁判相同）。本件原告因被告之過失傷害行為，致受有左側脛骨幹併腓骨幹粉碎開放性骨折、左側近端腓骨骨折併脫位及左側腓神經受損等傷勢，且經骨折手術治療，足認其身心受有相當大程度之痛苦，則原告請求被告賠償慰撫金，洵屬有據。本院審酌原告所受傷勢非輕，迄至111年7月25日骨頭仍尚未完全癒合（詳見下述），及原告為大學畢業，目前無業，名下無不動產或其他財產，110年度所得總額約108,688元；被告為國中畢業，110年度所得總額約22,074元，名下有坐落新北市蘆洲區房屋2筆、土地1筆暨坐落嘉義縣溪口鄉田賦1筆、汽車1輛，110年度財產總額約8,770,480元，此據兩造陳明在卷，且有兩造之稅務電子閘門財產所得調件明細表、另參酌兩造身分、經濟、社會地位、被告加害情形對原告造成之痛苦程度等一切情狀，仍認原告請求被告賠償慰撫金380萬元，尚屬過高，應核減為50萬元，始為適當。（七）預估復建費用部分：原告主張受傷後，經醫院預估復健時間至少要半年，須支出復健費用93,933元等情，雖未提出相關單據為佐證，惟經本院依職權就原告所受傷勢，於門診追蹤治療期間是否需經復健始能痊癒？如是，期間多長？該傷患又需自行負擔多少復健費用？等事項，向馬偕紀念醫院查詢，結果覆稱：「....方君因左側下肢粉碎性骨折，於109年11月4日開始於本院治療，至111年7月25日止，X光顯示骨折尚未癒合，復建治療可改善下肢肌耐力與活動度，有助痊癒，因骨頭尚未癒合，故無法評估需多久時間及金額估算」等情，此有該院111年9月26日馬院醫骨字第1110006173號函在卷可憑，故可認原告之傷勢於骨折癒合後，確有接受復健治療之必要，而原告主張骨折癒合後接受復健治療期間約半年，尚屬合理可信；再以一般醫院每3週看一次復健科門診，並進行6次復健方式復健，並以一次復健科門診費用平均約200元，復健6次花費300元計算，則原告於一次復健療程約須花費500元，亦屬合理，則以原告於復健半年時間約9個療程（計算式：180天÷21天≒9）計算，原告共約須支出復健費用4,500元（計算式：500元×9＝4,500元）。（八）以上合計，原告因被告之侵權行為所受損害共925,691元（計算式：4,345元＋113,576元＋198,000元＋15,090元＋90,180元＋50萬元＋4,500元＝925,691元）。五、綜上所述，原告依侵權行為之法律關係，請求被告給付925,691元，及自起訴狀繕本送達翌日即111年3月29日起至清償日止，按年息5%計算之法定遲延利息，為有理由，應予准許；逾此部分之請求，則無理由，應予駁回。六、本判決原告勝訴部分，係依簡易程序而為被告敗訴之判決，爰依職權宣告假執行。至於原告敗訴部分，其假執行之聲請，即失所附麗，應併予駁回。中華民國111年11月25日法官趙義德以上為正本係照原本作成。如不服本判決，應於送達後20日內，向本院提出上訴狀並表明上訴理由，如於本判決宣示後送達前提起上訴者，應於判決送達後20日內補提上訴理由書（須附繕本）。中華民國111年11月25日書記官張裕昌"
#mytext = "臺灣雲林地方法院民事判決104年度訴字第616號原告沈克威被告洪英豪上列被告因過失傷害案件，經原告提起損害賠償之附帶民事訴訟（104年度交簡附民字第13號），經本院刑事庭裁定移送前來，本院於民國105年6月29日言詞辯論終結，判決如下：主文被告應給付原告新臺幣壹拾參萬捌仟零肆拾壹元，及自民國一○四年八月二十七日起至清償日止，按週年利率百分之五計算之利息。原告其餘之訴駁回。本判決第一項得假執行。原告其餘假執行之聲請駁回。事實及理由壹、程序事項一、按訴狀送達後，原告不得將原訴變更或追加，但擴張或減縮應受判決事項之聲明者，不在此限，民事訴訟法第255條第1項第3款定有明文。原告起訴時原請求被告應給付原告新臺幣（下同）1,098,707元，及自起訴狀繕本送達翌日起至清償日止，按年息百分之5計算之利息（附民卷第2頁）。嗣於民國105年5月19日準備程序中當庭變更訴之聲明，請求被告應給付原告613,849元，及自起訴狀繕本送達翌日起至清償日止，按年息百分之5計算之利息（本院卷第127頁），為聲明之減縮，揆諸上開規定，並無不合，應予准許。二、本件被告經合法通知，未於言詞辯論期日到場，核無民事訴訟法第386條各款所列情形，爰依原告之聲請，由其一造辯論而為判決。貳、實體事項一、原告主張：被告於103年10月1日17時許，駕駛車牌號碼0000-00號自小貨車沿雲林縣大埤鄉豐田村某產業道路1（下稱道路1）由東往西方向行駛，嗣於同日17時44分許，行經道路1與該村某產業道路2（下稱道路2）之交岔路口時，本應注意汽車行經無號誌之交岔路口時，應減速慢行，作隨時停車之準備，而依當時情形為天候晴、暮光、柏油乾燥無缺陷路面及無障礙物等，並無不能注意之情事，其於通過該交岔路口前，已自該處之反光鏡內，發現伊騎乘車牌號碼000-000號普通重型機車，沿道路2由南往北向朝該交岔路口駛來，被告竟未暫停並採取適當措施而逕行通過該交岔路口，二車因而發生碰撞（下稱系爭事故），伊因而人車倒地，並受有右側髕骨粉碎性骨折之傷害，爰依侵權行為法律關係提起本件訴訟，請求被告賠償醫療費用6,517元、復健費用1,290元、看護費用28,000元、不能工作損失72,524元、購買營養品、四角椅及護膝用具費用共5,518元、精神慰撫金50萬元，總計613,849元，並聲明：㈠被告應給付原告613,849元，及自起訴狀繕本送達翌日起至清償日止，按年息百分之5計算之利息；㈡願供擔保請准宣告假執行。二、被告則以：原告之請求金額並不合理，且原告就系爭事故之發生亦負有過失責任，應予過失相抵等語，資為抗辯，並聲明：原告之訴駁回。三、兩造不爭執之事實：㈠被告於103年10月1日17時許，駕駛車牌號碼0000-00號自小貨車，沿雲林縣大埤鄉豐田村道路1由東往西方向行駛，嗣於同日17時44分許，行經道路1與該村道路2之交岔路口時，本應注意汽車行經無號誌之交岔路口時，應減速慢行，作隨時停車之準備，而依當時情形為天候晴、暮光、柏油乾燥無缺陷路面及無障礙物等，並無不能注意之情事，其於通過該交岔路口前，已自該處之反光鏡內，發現原告騎乘車牌號碼000-000號普通重型機車，沿道路2由南往北方向朝該交岔路口駛來，被告竟未暫停並採取適當措施而逕行通過該交岔路口，二車因而發生碰撞，原告因而人車倒地，並受有右側髕骨粉碎性骨折之傷害。㈡被告因業務過失傷害案件經本院104年度交簡字第68號刑事判決判處拘役40日，如易科罰金，以1,000元折算1日確定。㈢原告因系爭事故支出醫藥費用6,517元，至105年4月14日支出復健費用1,290元，購買營養品、四角椅、護膝用具等必要用品支出費用共5,518元。㈣原告因發生系爭事故需專人全日看護期間2週，支出看護費用28,000元。㈤原告發生系爭事故時，每月薪資為20,721元，不能工作期間為3個月2週，不能工作損失共計72,524元。四、得心證之理由：本件兩造所爭執之處，在於：㈠原告對於系爭事故之發生有無過失？過失比例為何？㈡原告得請求被告賠償之醫療費用、復健費用、必要用品支出費用、看護費用、不能工作損失及精神慰撫金金額為何？茲論述如下：㈠原告主張被告於103年10月1日17時許，駕駛車牌號碼0000-00號自小貨車，沿雲林縣大埤鄉豐田村道路1由東往西方向行駛，嗣於同日17時44分許，行經道路1與該村道路2之交岔路口時，本應注意汽車行經無號誌之交岔路口時，應減速慢行，作隨時停車之準備，而依當時情形為天候晴、暮光、柏油乾燥無缺陷路面及無障礙物等，並無不能注意之情事，其於通過該交岔路口前，已自該處之反光鏡內，發現原告騎乘車牌號碼000-000號普通重型機車，沿道路2由南往北方向朝該交岔路口駛來，被告竟未暫停並採取適當措施而逕行通過該交岔路口，二車因而發生碰撞，原告因而人車倒地，並受有右側髕骨粉碎性骨折之傷害，有臺大醫院雲林分院診斷證明書5紙及現場照片8張可稽（警卷第12至15頁、附民卷第20至24頁），且為被告所不爭執（本院卷第125至126頁），堪信為真實。而被告因業務過失傷害案件經本院104年度交簡字第68號刑事判決判處拘役40日，如易科罰金，以1,000元折算1日確定，業據本院調閱上開刑事卷宗核閱屬實。又按行經無號誌之交岔路口，應減速慢行，作隨時停車之準備；汽車行駛時，駕駛人應注意車前狀況，並隨時採取必要之安全措施，道路交通安全規則第93條第1項第2款、第94條第3項定有明文，被告駕駛車輛，自應注意遵守上開交通規定，且依系爭事故現場有暮光、天晴、柏油路面乾燥且無缺陷、無障礙物，此有現場事故調查報告表附卷可憑（警卷第9頁），並無不能注意之情事，而被告竟疏未注意車前狀況，亦未暫停並採取適當措施而逕行通過該交岔路口，而與原告所騎乘之車輛發生碰撞，致原告受有前揭傷害，其有過失至明，且被告之過失傷害行為與原告所受傷害之結果具相當因果關係，是原告主張被告應負侵權行為之責任，核屬有據。㈡按因故意或過失，不法侵害他人權利者，負損害賠償責任；不法侵害他人之身體或健康者，對於被害人因此喪失或減少勞動能力或增加生活上需要時，應負損害賠償責任；又不法侵害他人之身體、健康、名譽、自由、信用、隱私、貞操，或不法侵害其他人格法益而情節重大者，被害人雖非財產上之損害，亦得請求賠償相當之金額，民法第184條第1項前段、第193條第1項、第195條第1項分別定有明文。被告過失傷害原告之事實，已見前述，是被告依上述規定對原告所受之損害，自應負賠償責任。茲就原告據以請求被告賠償之項目及金額，應否准許，分述如下：⒈醫療費用、復健費用部分：原告主張因被告上開侵權行為支出醫療費用6,517元，及自系爭事故發生後至105年4月14日止支出復健費用1,290元，有原告之臺大雲林分院醫療費用收據、若瑟醫院診斷證明書、復健費用收據、若瑟醫院105年4月12日若瑟事字第0000000000號函及函附復健科醫療費用明細及收據等件為證（附民卷第5至15頁、本院卷第77至89頁、第111至113頁），並為被告所不爭執（本院卷第126頁），原告因被告之侵權行為，致增加此部分之生活上需要，是原告請求被告賠償醫療費用6,517元、復健費用1,290元，自屬有據。⒉購買營養品、四角椅、護膝用具等用品費用部分：原告主張因被告上開侵權行為支出購買營養品、四角椅、護膝用具等必要用品費用共5,518元，有護膝用具、助行器、營養品等支出收據可佐（附民卷第25至26頁），並為被告所不爭執（本院卷第126頁），原告因被告之侵權行為，致增加此部分之生活上需要，是原告請求被告賠償購買上開物品之費用5,518元，應屬有據。⒊看護費用部分：按親屬代為照顧被害人之起居，固係基於親情，但親屬看護所付出之勞力並非不能評價為金錢，雖因二者身分關係而免除被害人之支付義務，惟此種基於身分關係之恩惠，自不能加惠於加害人。故由親屬看護時雖無現實看護費之支付，仍應認被害人受有相當於看護費之損害，得向上訴人請求賠償，始符公平原則（最高法院94年台上字第1543號判決同此見解）。原告主張其因系爭事故受有前揭傷害，自103年10月1日發生系爭事故後2週期間，由訴外人即原告之父親看護，共計受有看護費用之損失28,000元等情，業據其於本院準備程序中陳述甚詳（本院卷第74、125頁），並有國立臺灣大學醫學院附設醫院雲林分院診斷證明書1紙為證（附民卷第20頁），足認依原告所受上開傷害，自103年10月1日起2週期間，有由專人全日看護照顧其日常生活起居之必要。另原告主張以每日2,000元計算看護費用，本院審酌原告前開主張與一般聘僱職業看護之市場行情相當，且為被告所不爭執（本院卷第126頁），應屬可採，是原告請求被告賠償2週之看護費用28,000元，洵屬有據，應予准許。⒋不能工作損失部分：原告主張其每月薪資為20,721元，其自103年10月1日發生車禍，於103年10月1日接受骨折復位鋼釘固定手術，休養3個月，自103年10月2日至104年1月2日，3個月無法工作，再於104年6月24日接受拔釘手術，休養2週，自104年6月24日至104年7月8日，2週無法工作，不能工作期間共計3個月2週，不能工作損失共計72,524元，有德聿佳工業股份有限公司薪資所得證明、103年度各類所得扣繳暨免扣繳憑單、臺大醫院雲林分院診斷證明書可稽（附民卷第18至20、23、24頁），並為被告所不爭執（本院卷第126頁），堪信屬實。準此，原告請求被告賠償不能工作損失72,524元，為有理由，應予准許。⒌精神慰撫金部分：按法院酌定慰撫金額時，應斟酌加害人與被害人雙方之身分、資力與加害程度，及其他各種情形核定之；慰撫金是否相當，應以加害行為之加害程度及被害人所受痛苦，斟酌加害人及被害人之身分、經濟地位等各種情形定之（最高法院47年台上字第1221號、51年台上字第223號判例意旨參照）。原告因系爭事故因而受有右側髕骨粉碎性骨折之傷害，其身體及精神均受有相當之痛苦，並參以被告之過失程度等，被告之不法侵害行為應屬情節重大，應堪認定。本院審酌原告為大學畢業，從事擔任作業員；被告為高職畢業，擔任臨時工，月收入約18,000元等情，業據兩造陳明在卷（本院卷第60頁），並有稅務電子閘門財產所得調件明細表在卷可參（本院卷第37至47頁），及兩造身分、社會地位、智識水準及原告所受傷害程度等一切情形，認原告請求被告賠償精神慰撫金50萬元，尚屬過高，應核減為30萬元，始屬公允。⒍準此，原告因被告之侵權行為所受損害而得請求被告賠償之金額應為413,849元（計算式：6,517＋1,290＋5,518＋28,000＋72,524＋300,000＝413,849元）。㈢惟按，損害之發生或擴大，被害人與有過失者，法院得減輕賠償金額或免除之，民法第217條第1項定有明文。又按，行至無號誌或號誌故障而無交通指揮人員指揮之交岔路口，支線道車應暫停讓幹線道車先行。未設標誌、標線或號誌劃分幹、支線道者，少線道車應暫停讓多線道先行；車道數相同時，轉彎車應暫停讓直行車先行；同為直行車或轉彎車者，左方車應暫停讓右方車先行。但在交通壅塞時，應於停止線前暫停與他方雙向車輛互為禮讓，交互輪流行駛，道路交通安全規則第102條第1項第2款定有明文。查，系爭事故地點為無號誌交岔路口，且系爭事故現場有暮光、天晴、柏油路面乾燥且無缺陷、無障礙物，此有現場照片及現場事故調查報告表附卷可憑（警卷第9、12至15頁），而原告與被告於行經上開交岔路口均係直行，則依上揭道路交通安全規則第102條第1項第2款規定，系爭事故發生地點既未劃分幹、支線道，車道數又相同，且同為直行車，則路權優先之判斷端視何方為右方車。本件原告為左方車，被告為右方車，依上開規定，位於上開路口左方之原告自應禮讓右方之被告先行，及依當時之客觀情形亦無不能注意之情形，亦可知原告違反應暫停禮讓右方車之交通規則亦有過失。本院審酌系爭事故因原告未依規定禮讓右方之被告車輛先行，致被告難以預測原告禮讓與否而肇生，認原告於應為肇事主因。而系爭事故肇事責任經送請交通部公路總局嘉雲區車輛行車事故鑑定會鑑定結果，認「一、原告駕駛普通重型機車，行經無號誌交岔路口，左方車未暫停讓右方車先行，為肇事主因。二、被告駕駛自用小貨車，行經無號誌交岔路口，未減速慢行，作隨時停車之準備且預見狀況，未妥採安全措施，為肇事次因。」亦同此認定，有交通部公路總局嘉雲區車輛行車事故鑑定會鑑定意見書可參（偵字卷第11至12頁）。而被告疏未注意車前狀況，未暫停並採取適當措施而逕行通過該交岔路口，而與原告所騎乘之車輛發生碰撞，致原告受有前揭傷害，亦有過失，已如前述，本院審酌兩造之過失情形，認系爭事故發生之責任歸屬，被告之過失程度為百分之四十，原告之過失程度為百分之六十，應以原告過失程度之比例，減輕被告應負之賠償責任，依此過失比例計算後，原告得請求被告賠償之金額為165,540元（計算式：413,849×40％＝165,540元，元以下四捨五入）。㈣按汽車交通事故發生時，請求權人因事故汽車為未保險汽車，未能依本法規定向保險人請求保險給付者，得於本法規定之保險金額範圍內，向特別補償基金請求補償；特別補償基金依第40條規定所為之補償，視為損害賠償義務人損害賠償金額之一部分；損害賠償義務人受賠償請求時，得扣除之，強制汽車責任保險法第40條第1項第2款、第42條第1項分別定有明文。經查，原告於系爭事故發生後已經領取特別補償基金27,499元，有國泰世紀產物保險股份有限公司函文附卷可參（本院卷第137頁），是依上開規定，被告應賠償原告之數額自應扣除原告已領取之補償金。經依上開規定扣除原告已領取之特別補償基金後，原告得請求被告賠償之金額為138,041元（計算式：165,540－27,499＝138,041元）。㈤末按給付有確定期限者，債務人自期限屆滿時起，負遲延責任；給付無確定期限者，債務人於債權人得請求給付時，經其催告而未為給付，自受催告時起，負遲延責任；其經債權人起訴而送達訴狀，或依督促程序送達支付命令，或為其他相類之行為者，與催告有同一之效力；遲延之債務，以支付金錢為標的者，債權人得請求依法定利率計算之遲延利息，民法第229條第1、2項、第233條第1項分別規定甚明。原告請求自起訴狀繕本送達日翌日即自104年8月27日（附民卷第30頁）起至清償日止，按年息百分之5計算之利息，依上開規定亦有理由，應准許之。四、綜上所述，原告依侵權行為之法律關係，請求被告給付138,041元，及自起訴狀繕本送達翌日即104年8月27日起至清償日止，按年息百分之5計算之利息，為有理由，應予准許；逾此部分之請求，則無理由，應予駁回。五、經核原告勝訴部分，因所命給付之金額未逾50萬元，爰依民事訴訟法第389條第1項第5款之規定，依職權宣告假執行，此部分雖經原告陳明願供擔保請准宣告假執行，惟其聲請不過促請法院職權發動，本院無庸就其聲請為准駁之裁判，附此敘明。另原告敗訴部分，其假執行之聲請失所附麗，應予駁回。六、本件事證已臻明確，原告其餘主張及所提證據，經審酌與本院前開判斷不生影響，爰不再予以逐一論述；另本件係原告於刑事訴訟程序中附帶提起之民事損害賠償事件，經本院刑事庭依刑事訴訟法第504條第1項規定裁定移送前來，而依同條第2項規定刑事附帶民事訴訟本無徵收裁判費，且迄至本院言詞辯論終結止，亦無其他費用支出，是本件既無訴訟費用支出，故毋庸為訴訟費用負擔之諭知，併此敘明。七、據上論結，原告之訴為一部有理由，一部無理由，依民事訴訟法第389條第1項第5款，判決如主文。中華民國105年7月13日民事第一庭審判長法官冷明珍法官楊昱辰法官王靜慧以上正本係照原本作成。如對本判決上訴，須於判決送達後20日內向本院提出上訴狀。如委任律師提起上訴者，應一併繳納上訴審裁判費。中華民國105年7月13日書記官金雅芳"
#mytext = "臺灣臺北地方法院新店簡易庭民事判決110年度店簡字第1307號原告劉福華住桃園市○○區○○路0000巷00弄00街0號訴訟代理人董郁琦律師複代理人李思慧律師被告洪于斌住臺北市○○區○○○路○段000巷00號0樓訴訟代理人林茂淳複代理人張逸群同上上列當事人間損害賠償事件，本院於民國111年8月4日言詞辯論終結，判決如下：主文被告應給付原告新臺幣壹佰陸拾參萬捌仟陸佰參拾肆元，及自民國一百十年十一月十二日起至清償日止，按週年利率百分之五計算之利息。原告其餘之訴駁回。訴訟費用由被告負擔百分之四十五，餘由原告負擔。本判決原告勝訴部分得假執行。但被告如以新臺幣壹佰陸拾參萬捌仟陸佰參拾肆元為原告預供擔保後，得免為假執行。原告其餘假執行之聲請駁回。事實及理由壹、程序事項按因侵權行為涉訟者，得由行為地之法院管轄，民事訴訟法第15條第1項定有明文。查本件侵權行為地為新北市坪林區（見本院卷第21頁），本院自有管轄權。貳、實體事項一、原告主張：被告洪于斌於民國109年11月5日凌晨1點42分許，駕駛車牌號碼000-0000號自用小客車，沿新北市坪林區北宜公路由坪林往宜蘭方向行駛，行經該路段53公里處時，違規跨越雙黃線逆向駛入來車道，與伊駕駛車牌號碼000-00號營業大貨車（下稱系爭車輛）發生碰撞，並造成系爭車輛及其上載運雞隻失火焚燬，伊受有左膝挫傷併膝關節瘀血。伊已受系爭車輛所有權人建德貨運股份有限公司（下稱建德公司），與重置雞籠費用之柏威物流有限公司（下稱柏威公司）讓與相關債權，依侵權行為及債權讓與之法律關係，請求被告賠償如附表一所示內容，並聲明：㈠被告應給付原告新臺幣（下同）367萬5,475元元及自民事補充理由一狀繕本送達之翌日起至清償日止，按週年利率5％計算之利息；㈡願供擔保，請准宣告假執行（見本院卷第313、331頁）。二、被告則以：伊對雞隻清運與焚化、拖吊費用均不爭執，惟購置雞籠費用應計算折舊，系爭車輛交易價額應提出鑑價資料，而原告主張車上毀損物品損害147,490元，未提出證據證明，請求營業損失未扣除相關成本，另其主張慰撫金額過高等語置辯，並聲明：㈠原告之訴及其假執行駁回。㈡如受不利判決，願供擔保請准宣告免為假執行。三、法院之判斷：㈠按因故意或過失，不法侵害他人之權利者，負損害賠償責任；民法第184條第1項前段定有明文。汽車在未劃設慢車道之雙向二車道行駛時，應依下列規定：…二、在劃有分向限制線之路段，不得駛入來車之車道內。道路交通安全規則第97條第1項第2款定有明文。查被告於上開時、地駕駛車輛，不依規定駛入來車道，撞擊系爭車輛受損並致原告受傷等情，有原告提出交通事故初步分析研判表、道路事故現場圖、新聞報導、原告傷勢照片在卷可按（見本院卷第21至29頁），復被告到庭不爭執，堪信為真實，是被告過失行為與原告受傷及系爭車輛毀損結果間，具有相當因果關係。㈡按不法毀損他人之物者，被害人得請求賠償其物因毀損所減少之價額，民法196條定有明文，且依最高法院77年5月17日第9次民事庭總會決議見解，固認上開法條所指之物因毀損所減少之價額，得以修復費用為估定之標準，然其亦指出需以必要者為限，是若車輛受損後雖能修復，但如修理所需費用超過該車受損前之價格，則不能認為該修復費用係屬必要，自應以車輛受損前之價格，扣除車輛受損後之殘值，以其間差額作為損害額，始符誠信公平原則。查原告請求如附表一編號1所示內容，審酌系爭車輛於103年1月間出廠使用，有行車執照及車籍資料在卷可按（見本院卷第37、77頁），迄至車禍發生之109年11月5日已使用6年餘，系爭車輛被撞後，車頭及車體均嚴重受損變形，估定修復金額高達291萬2,896元，有原告提出火燒車照片及長源汽車估價單在卷足參（見本院卷第31至35頁），堪認系爭車輛於被撞後已達回復原狀顯有重大困難程度。復依原告提出中華民國汽車鑑價協會111年7月25日函，系爭車輛在車況正常保養情形良好下，於109年11月間市場交易價格為150萬元（不含風扇、灑水設備）等語（見本院卷第327頁），又系爭車輛殘值為85,000元，有原告提出鼎巨環保回收有限公司－報廢車估價單在卷可按（見本院卷第2257頁），依上開資料，本院認為系爭車輛於本件侵權行為發生所減少價額應以141萬5,000元（150萬元－85,000元＝141萬5,000元）為適當，逾上開範圍部分，應予駁回。㈢按當事人主張有利於己之事實者，就其事實有舉證之責任。民事訴訟法第277條前段定有明文。又民事訴訟如係由原告主張權利者，應先由原告負舉證之責，若原告先不能舉證，以證實自己主張之事實為真實，則被告就其抗辯事實即令不能舉證，或其所舉證據尚有疵累，亦應駁回原告之請求（最高法院104年度臺上字第41號判決意旨參照）。查原告主張如附表一編號2所示內容，審酌原告提出中華民國汽車鑑價協會110年11月29日函文內容，系爭車輛在車況正常保養情形良好下，於109年11月間市場交易價格為155萬元（含風扇、灑水設備）等語（見本院卷第161至163頁），嗣該會111年7月25日函稱：系爭車輛不含風扇、灑水設備之交易價格為150萬元等語（見同上），可知系爭車輛上確有風扇、灑水設備且車禍發生時市場交易價格5萬元，原告此部分請求洵屬有據。至於附表二所示其餘請求，原告雖稱：法院得依民事訴訟法第222條第2項規定，審酌一切情況依所得心證定其數額等語；惟此項規定，性質上乃證明度之降低，而非純屬法官之裁量權，負舉證責任之當事人仍應在客觀上可能之範圍內提出證據，是在損害賠償之訴，被害人已證明受有損害，有客觀上不能證明其數額或證明顯有重大困難之情事時，如仍強令被害人舉證證明損害數額，非惟過苛，亦不符訴訟經濟原則之故，才由法院審酌一切情況定其數額。原告自應舉證證明系爭車輛上有哪些物品確因失火受損，惟其未能舉證證明除風扇、灑水設備外其他物品亦有損失，原告逾此5萬元範圍之請求，則無理由，應予駁回。㈣依前揭最高法院77年5月17日第9次民事庭會議決議，債權人所得請求者既為回復原狀之必要費用，倘以修復費用為估定其回復原狀費用之標準，則修理材料以新品換舊品時，即應予折舊。查原告請求如附表一編號3所示內容，提出統一發票在卷（見本院卷第63頁），本院審酌行政院所頒固定資產耐用年數表，運輸業用客車、貨車之耐用年數為4年，其上雞籠屬供原告運輸使用，可比照行政院所頒固定資產耐用年數表所載運輸業用貨車之耐用年數4年加以計算扣除折舊額，而系爭車輛計算至車禍發生時，已使用逾前述汽車耐用年數4年，應僅存殘值，再依所得稅法第51條第1項及該法施行細則第48條第1款規定，以平均法計算其殘值（即以固定資產成本減除殘價後之餘額，按固定資產耐用年數表規定之耐用年數平均分攤，計算每期折舊額），則扣除折舊後之價值為15,984元【計算式：殘價＝取得成本÷（耐用年數＋1）＝79,920元÷5＝15,984元】。㈤按負損害賠償責任，除法律另有規定或契約另有訂定外，應回復他方損害發生前之原狀，前項情形，債權人得請求支付回復原狀所必要之費用，以代回復原狀，第213條定有明文。查原告請求如附表一編號4至5所示內容，提出清運雞隻費用、拖吊費用之統一發票（見本院卷第65至67頁），復被告到庭不爭執（見本院卷第105頁），此部分金額應屬有據。㈥原告主張自109年11月至110年2月因系爭車輛毀損無法載送貨物，擔任受僱司機月薪3萬餘元，比照109年5月至10月間為柏威公司載運貨物月平均營業額199,461元，其受有營業損失650,415元（參見如附表三）等語，為被告所否認，揆諸前揭舉證責任分配原則，原告自應就其受有上開營業損失負舉證責任；惟查：原告並無在系爭車輛受損期間停業，有原告提出柏威公司薪資單在卷可按（見本院卷第59至61頁），原告主張因此受有營業損失，雖提出柏威公司出具報酬明細及毛雞運輸承包合約書在卷（見本院卷第45至57、191至194頁），然就該段期間營業收入（見本院卷第13頁），尚未扣除加油費、行費、通行費、車輛保養費及各式稅費，為原告陳明在卷併提出相關單據可參（見本院卷第268至299頁），足見原告計算營業利潤方式並未考量營業成本等因素，另原告主張自109年11月至110年2月共計4個月無法營業，重置車輛期間顯然過長且未提出任何資料供參，又系爭車輛所有權人為建德公司，有前揭行照及車籍查詢在卷可按（見同上），原告雖稱其為實際所有人，惟於本院言詞辯論終結前，均未提出與建德公司間自備車輛參與經營契約書供參，尚無法逕自認定為原告所有，再依原告提出柏威公司與建德公司之債權讓與書內容，建德公司主張其所有之車輛毀損嚴重受有損害，額外支出拖吊費、清運、焚化雞隻等費用，對被告有179萬7,650元之債權本金等語（見本院卷第211頁），並未包括營業損失，此部分營業損失縱然屬實亦屬建德公司，與原告因發生車禍所衍生損害，無相當因果關係，原告此部分之請求，不應准許。㈦按不法侵害他人之身體、健康、名譽、自由、信用、隱私、貞操，或不法侵害其他人格法益而情節重大者，被害人雖非財產上之損害，亦得請求賠償相當之金額。民法第195條第1項有明文。慰藉金之賠償須以人格權遭遇侵害，使精神上受有痛苦為必要，其核給之標準固與財產上損害之計算不同，然非不可斟酌雙方身分資力與加害程度，及其他各種情形核定相當之數額。其金額是否相當，自應依實際加害情形與被害人所受之痛苦及雙方之身分、地位、經濟狀況等關係決定之（最高法院51年臺上字第223號、85年度臺上字第460號裁判意旨參照）。查原告因被告過失行為受有前述傷害，提出佑群骨科診所診斷證明書在卷可按（見本院卷第209頁），精神上自受有相當痛苦。本院衡酌兩造資力，有稅務電子閘門資料查詢表在卷可憑（置於限閱卷），以及被告之駛入來車道之過失程度、原告受傷情形等一切情狀，認原告請求被告賠償精神慰撫金應以6萬元為適當，逾此部分之請求，難認有據，無法准許。㈧按給付無確定期限者，債務人於債權人得請求給付時，經其催告而未為給付，自受催告時起，負遲延責任，其經債權人起訴而送達訴狀者，與催告有同一之效力；遲延之債務，以支付金錢為標的者，債權人得請求依法定利率計算之遲延利息；應付利息之債務，其利率未經約定，亦無法律可據者，週年利率為百分之5，民法第229條第2項、第233條第1項前段、第203條分別定有明文。查原告對被告之侵權行為損害賠償債權，核屬無確定期限之給付，既經原告提起訴訟且民事補充理由一狀繕本，業於110年11月11日當庭交付被告（見本院卷第149頁），被告迄未給付，應負遲延責任，是原告請求自該書狀繕本送達被告之翌日即110年11月12日起至清償日止，按週年利率5％計算之法定遲延利息，核無不合，應予准許。四、綜上所述，原告依據侵權行為之法律關係，請求被告給付163萬8,634元（141萬5,000元＋5萬元＋15,984元＋47,250元＋50,400元＋6萬元＝163萬8,634元）及自110年11月12日起至清償日止，按週年利率5％計算之法定遲延利息，為有理由，應予准許，逾此部分之請求，則屬無據，應予駁回。其假執行之聲請，因駁回而失所附麗，併予駁回。五、本件事證已臻明確，兩造其餘攻擊防禦方法及所提證據，核均與本件判決結果不生影響，爰不一一論述，併予敘明。六、本件原告勝訴部分，係就民事訴訟法第427條第1項訴訟適用簡易程序所為被告敗訴之判決，依同法第389條第1項第3款規定，應依職權宣告假執行，並依同法第436條第2項適用第392條第2項規定，依職權宣告被告如預供擔保，得免為假執行。七、訴訟費用負擔之依據：民事訴訟法第79條。中華民國111年9月16日臺灣臺北地方法院新店簡易庭法官鄧德倩上列正本證明與原本無異。如不服本判決，應於判決送達後20日內向本庭提出上訴狀（須按他造當事人之人數附繕本）；如委任律師提起上訴者，應一併繳納上訴審裁判費。中華民國111年9月16日書記官張嘉崴附表一：原告請求項目及金額（見本院卷第316頁）編號請求項目金額（新臺幣，下同）主張內容1系爭車輛市價170萬元受損嚴重已無修復實益，請求事故發生之價額2車上毀損物品147,490元細目見附表二3雞籠整理箱79,920元雞籠單價555元×144個毀損＝79,920元4雞隻清運、焚化47,250元5拖吊費50,400元從事發地點拖吊回公司6營業損失650,415元原告本可獲得月平均營業額199,461元，因系爭車輛毀損無法從事承攬運送業務，擔任受僱司機月薪3萬餘元，請求109年11月至110年2月差額（細目見附表三）7精神慰撫金100萬元總計：367萬5,475元附表二：車上毀損物品細目（見本院卷第12、186至188頁）後車斗（參考市價見本院卷第203至204頁網路資料）編號項目請求金額參考市價安裝照片示意圖1白鐵電風扇18,000元一支單價4,800元，4支合計19,200元車上原裝有4支，見本院卷第43、199頁。2抽水機11,000元11,000元見本院卷第41、200頁。3水箱20,400元20,400元（參東興板金工業社估價單）見本院卷第43、199頁。4水管3,500元4,000元（每公尺80元，總長50公尺）見本院卷第41、201頁。5發電機26,400元25,460元（發電機安裝較為困難，加估安裝費用940元）見本院卷第41、200頁。6押板3,200元3,200元（單價估400元，共組裝8組）7繩索9,500元一捆單價1,080元，10捆合計10,800元小計92,000元車頭（參考市價見本院卷第205至207頁網路資料）編號項目請求金額參考市價主張內容8皮包1,300元1,549元（890+599+60=1,549）原告車上皮包含小皮包與背包，合計兩種並加計運費60元9現金3,500元3,500元據原告印象當時仍有3,500元10護腰1,620元1,699元11ETC200元200元政府規費200元12駕照200元200元政府規費200元13健保卡200元200元政府規費200元14遙控器7,000元6,726元參見原告另一遙控器單據15定速器18,500元17,100元（17,000+100=17,100）加計運費100元16車機18,000元19,000元17GPS充電器600元674元（599+75=674）加計運費75元18充電手電筒500元719元（569+150=719）加計運費150元19坐墊720元789元（729+60=789）加計運費60元20音響主機950元1,235元（1,035+200=1,235）加計運費200元21方向盤皮套800元1,193元（893+300=1,193）加計運費300元。22藍芽1,400元1,560元（750×2+60=1,560）原告車上裝有二支藍芽音樂播放器並加計運費60元。小計55,490元總計147,490元附表三：營業損失計算內容（見本院卷第13至14頁）月份原可取得報酬實領薪資差額109年11月199,461元35,429元164,032元109年12月199,461元30,400元169,061元110年1月199,461元49,600元149,861元110年2月199,461元32,000元167,461元合計650,415元109年5月至10月平均報酬199,461元計算方式如下：（189,621元＋176,000元＋205,310元＋216,657元＋196,718元＋212,457元）÷6個月＝199,461元"
print(len(mytext))

4472


### Test fine-tune model

In [13]:

from pprint import pprint
from paddlenlp import Taskflow
my_ie = Taskflow("information_extraction", schema=schema, 
                 task_path='./data_v3/checkpoint/model_best',
                 precision='fp32')
pprint(my_ie(mytext))

#fp16 -> pip install onnxruntime-gpu onnx onnxconverter-common


[2023-03-21 05:43:58,809] [    INFO] - loading configuration file ./data_v3/checkpoint/model_best/config.json
[2023-03-21 05:43:58,810] [    INFO] - Model config ErnieConfig {
  "architectures": [
    "UIE"
  ],
  "attention_probs_dropout_prob": 0.1,
  "dtype": "float32",
  "enable_recompute": false,
  "fuse": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "ernie",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "paddlenlp_version": null,
  "pool_act": "tanh",
  "task_id": 0,
  "task_type_vocab_size": 3,
  "type_vocab_size": 4,
  "use_task_id": true,
  "vocab_size": 40000
}

[2023-03-21 05:44:00,108] [    INFO] - All model checkpoint weights were used when initializing UIE.

[2023-03-21 05:44:00,109] [    INFO] - All the weights of UIE were initialized from the model checkpoint at ./dat

[{'原告': [{'text': '陳柏村', 'start': 27, 'end': 30, 'probability': 0.9985266957727177, 'relations': {'法院得心證判斷之適當裁定精神慰撫金': [{'text': '6萬元', 'start': 3732, 'end': 3735, 'probability': 0.6164677174951407}], '非財產上之損害的求償精神慰撫金': [{'text': '70萬元', 'start': 894, 'end': 898, 'probability': 0.5161447176547505}, {'text': '70萬元', 'start': 2792, 'end': 2796, 'probability': 0.37009963632714005}]}}, {'text': '陳柏村', 'start': 27, 'end': 30, 'probability': 0.9985266957727177}, {'text': '陳柏村', 'start': 27, 'end': 30, 'probability': 0.9985266957727177}], '法院得心證判斷之適當裁定精神慰撫金': [{'text': '70萬元', 'start': 2792, 'end': 2796, 'probability': 0.2967277825738108}, {'text': '6萬元', 'start': 3732, 'end': 3735, 'probability': 0.8513134822176198}], '非財產上之損害的求償精神慰撫金': [{'text': '70萬元', 'start': 894, 'end': 898, 'probability': 0.6604962323243058}, {'text': '70萬元', 'start': 2047, 'end': 2051, 'probability': 0.7195688154567961}, {'text': '70萬元', 'start': 2792, 'end': 2796, 'probability': 0.8753819682443549}, {'text': '70萬元', 

### Test non-fine-tune model

In [4]:
from pprint import pprint
from paddlenlp import Taskflow
ie = Taskflow('information_extraction', schema=schema, model="uie-base")
pprint(ie(mytext))

[2023-03-16 06:44:23,089] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load '/home/ubuntu/.paddlenlp/taskflow/information_extraction/uie-base'.


[{'其他精神慰撫金': [{'end': 472,
               'probability': 0.6937654440685392,
               'start': 468,
               'text': '14萬元'}],
  '原告': [{'end': 36,
          'probability': 0.8224311801136963,
          'relations': {'法院得心證判斷之適當裁定精神慰撫金': [{'end': 472,
                                               'probability': 0.495825265987083,
                                               'start': 468,
                                               'text': '14萬元'}],
                        '非財產上之損害的求償精神慰撫金': [{'end': 472,
                                             'probability': 0.7485344173362805,
                                             'start': 468,
                                             'text': '14萬元'}]},
          'start': 33,
          'text': '楊健成'},
         {'end': 36,
          'probability': 0.8224311801136963,
          'start': 33,
          'text': '楊健成'},
         {'end': 36,
          'probability': 0.8224311801136963,
          'start': 33,
          'text'

In [10]:
import numpy as np
import pandas as pd
test = np.array([1,33, 2, 19,34])


a = {}
a["1"] = (0, "haha")
a["22"] = (12, "gg")
a["333"] = (1234, "haha")
#print(a)

print(list(a.items()))


[('1', (0, 'haha')), ('22', (12, 'gg')), ('333', (1234, 'haha'))]


In [6]:


test = "臺灣士林地方法院民事簡易判決110年度簡字第75號原告甲女（真實姓名年籍詳卷）兼法定代理倪秀宜人共同訴訟代理人林靜文律師（法扶律師）被告徐若如訴訟代理人黃教倫律師上列當事人間請求損害賠償事件，原告提起刑事附帶民事訴訟，經本院刑事庭裁定移送前來（108年度審交附民字第426號），本院於民國110年3月10日言詞辯論終結，判決如下：主文被告應給付原告甲女新臺幣玖仟捌佰玖拾元，及自民國一百零八年十月九日起至清償日止，按週年利率百分之五計算之利息。被告應給付原告甲○○新臺幣肆拾玖萬玖仟叁佰叁拾玖元，及自民國一百零八年十月九日起至清償日止，按週年利率百分之五計算之利息。原告其餘之訴駁回。訴訟費用由被告負擔百分之三十五，餘由原告負擔。本判決第一項於原告甲女以新臺幣叁仟貳佰玖拾柒元為被告供擔保後，得假執行。但被告以新臺幣玖仟捌佰玖拾元為原告甲女預供擔保，得免為假執行。本判決第二項於原告甲○○以新臺幣壹拾陸萬陸仟肆佰肆拾陸元為被告供擔保後，得假執行。但被告以新臺幣肆拾玖萬玖仟叁佰叁拾玖元為原告甲○○預供擔保，得免為假執行。原告其餘假執行之聲請駁回。事實及理由壹、程序部分一、按民事訴訟法第427條第2項於民國110年1月20日修正公布，新增第11款「本於道路交通事故有所請求而涉訟」之訴訟，不問其標的金額或價額一律適用簡易程序，並自110年1月22日起施行。又上開修正條文施行後，於修正前已繫屬之事件，其法院管轄權及審理程序，未經終局裁判者，適用修正後之規定，曾經終局裁判者，則適用修正前之規定，民事訴訟法施行法亦增訂第4條之1規定甚明。本件原告係本於道路交通事故請求侵權行為損害賠償，核屬修正後民事訴訟法第427條第2項第11款所列之訴訟，於修正前已繫屬本院，尚未經終局裁判，依民事訴訟法施行法第4條之1規定，其審理程序應適用修正後之規定，故本件爰依簡易程序為民事簡易判決，合先敘明。二、次按簡易訴訟訴狀送達後，原告不得將原訴變更或追加他訴。但擴張或減縮應受判決事項之聲明者，不在此限。民事訴訟法第436條第2項、第255條第1項第3款分別定有明文。查原告提起附帶民事訴訟時，原聲明請求被告給付原告甲女新臺幣（下同）21萬3,648元、原告甲○○218萬4,741元，及均自刑事附帶民事訴訟起訴狀繕本送達翌日起至清償日止，按年利率百分之五計算之利息。嗣原告於審理中先後擴張、減縮聲明，其終聲明為：㈠被告應給付原告甲女21萬3,648元，及自起訴狀繕本送達翌日起至清償日止按週年利率百分之五計算之利息。㈡被告應給付原告甲○○123萬371元，及自起訴狀繕本送達翌日起至清償日止按年息百分之五計算之利息。㈢願供擔保請准宣告假執行（見本院卷第204頁民事綜合辯論意旨狀）。經核原告所為係擴張、減縮應受判決事項之聲明，揆諸上開說明，自應准許。貳、實體部分一、原告主張：㈠被告於107年10月23日上午7時46分時，將車牌號0000-00自用小客車（下稱系爭自小客車），停放於新北市○○區○○街000號旁之停車格內，本應注意開啟車門時，應確認後方有無行駛車輛，竟疏未注意及此，即貿然開門下車，適有原告甲○○騎乘車牌號000-000號重型機車（下稱系爭機車）搭載其女即原告甲女，沿新北市汐止區明峰街往臺北方向行駛至該處，致人車倒地（下稱系爭交通事故），原告甲○○因而受有右手中指近端指骨骨折、胸部挫傷、頭部外傷並腦震盪等傷害，原告甲女因而受有左肘、右膝及左臉挫傷、頭部外傷病腦震盪等傷害（下稱系爭傷害）。㈡原告2人因系爭交通事故受有之損害如下：⒈醫療費用：原告甲女支出3,648元、原告甲○○7,234元。⒉增加生活支出：⑴原告甲女因系爭傷害，有嘔吐回診無法上學之情形，因必須觀察，醫院建議休養一個月；原告甲○○則因系爭傷害，多次接受骨折之鋼釘固定手術，原告甲○○之配偶（原告甲女父親）因而必須請假照顧原告2人，故原告甲女、甲○○分別以30日、5日每日2,000元計算，分別受有6萬元及1萬元之看護費用損失。⑵原告甲○○自受傷以來，為加速復原，購買補充鈣質、甲殼素等營養品，共計3萬7,350元。⒊薪資及減少勞動能力之損失：原告甲○○因系爭交通事故受傷後，向公司請假多日，遭扣薪水三個月計3萬9,840元；又其原年收入為45萬9,184元因系爭交通事故造成右手中指指股骨折，經治療後仍無法復原，右手中指遠端指節喪失生活功能達二分之一以上，可活動角度剩零。而原告甲○○原於係擔任抄表作業員，必須用右手全手翻動成品，因右手中指功能減損而無法做原來之工作，致必須減薪調職，依臺北榮民總醫院之評估報告，原告甲○○主張勞動能力減損程度10%，以原告一年薪資計算，原告甲○○一年之損害額為10萬5,934元（459184×0.2307≒105934），原告甲○○現年為38歲，至其退休年齡65歲尚有27年，故原告甲○○得請求勞動能力減損之金額為77萬1,633元。⒋系爭機車維修費用支出1萬4,260元。⒌精神慰撫金：原告2人因系爭交通事故受分別受有系爭傷害，均痛苦不堪，且原告甲○○因勞動力減損，影響工作及生活能力，被告亦無和解誠意，致原告甲女及甲○○身心俱疲，精神上痛苦無比，爰分別請求非財產上損害賠償15萬元及35萬元。㈢為此依民法第184條第1項前段、第193條第1項、第195條規定提起本訴等語，並聲明：⒈被告應給付原告甲女21萬3,648元，及自起訴狀繕本送達翌日起至清償日止按週年利率百分之五計算之利息。⒉被告應給付原告甲○○123萬317元，及自起訴狀繕本送達翌日起至清償日止按年息百分之五計算之利息。⒊願供擔保請准宣告假執行。二、被告則以：㈠對系爭交通事故發生經過及原告2人受有系爭傷害等情不爭執，惟就原告2人請求損害賠償之數額，意見如下：⒈醫療費用部分：依三總醫院109年9月9日函覆內容可知，原告甲女疑似中樞神經免疫系統發炎之相關診斷與系爭交通事故無關，再參酌原告檢附之108年2月1日診斷證明書及三軍總醫院109年10月30日函覆內容，其主要住院原因為疑似中樞神經免疫系統發炎之復發，且後續亦因疑似中樞神經免疫系統發炎之可能復發回診持續追蹤，足見，原告甲女於108年1月21日以後在三軍總醫院小兒科門診或住院之醫療費用，係因疑似中樞神經免疫系統發炎相關之醫療費用，與本件事故無關，被告自無須負賠償之責；其餘部分及原告甲○○之醫療費用不爭執。⒉增加生活支出部分：⑴同前述，醫院建議原告甲女宜休養一個月居家照護一個月，係因上開病症所致，故原告甲女請求看護費用6萬元云云，顯然無據；又本件原告雖提出診斷證明書，主張原告甲○○有專人看護必要，且由其配偶請假照顧云云，惟原告甲○○之配偶是否確有請假照顧原告，尚非無疑，則原告甲○○自有其夫請假之相關證明，以盡其舉證責任。⑵原告甲○○因購買補充鈣質、甲殼素等營養品，共支出3萬7,350元，然此費用係原告甲○○未經醫師指示擅自購買服用，似非必要費用，被告自無須負賠償之責。⒊薪資及減少勞動能力之損失部分：原告甲○○雖主張其右手中指喪失生活功能2分之1以上，然參酌臺北榮民總醫院函文，原告甲○○工作功能受限約為1%至10%，故原告甲○○減損之勞動能力應係1%，較為公允。⒋系爭機車維修費用：原告甲○○雖請求機車維修費用1萬4,260元，然提出之修復單據僅記載機車零件，並未詳列修復項目，無從證明修復項目是否與系爭交通事故有關，且亦未按機車年份計算折舊，亦有不當。⒌慰撫金部分：原告甲女、甲○○分別請求15萬元及35萬元顯然過高，衡情應分別以1萬元、3萬元為適當。㈡此外原告甲女、甲○○因系爭交通事故已獲理賠強制汽車責任險保險金110元及14萬7,194元，且原告2人於刑案調解時再分別獲被告賠償3萬元及7萬元等語資為抗辯。並聲明：⒈原告之訴駁回。⒉如受不利判決，願供擔保請求免為假執行。三、本院之判斷：㈠原告主張被告於107年10月23日上午7時46分時，將系爭自小客車停放於新北市○○區○○街000號旁之停車格內，於開啟車門時，適有原告甲○○騎乘系爭機車搭載原告甲女，沿新北市汐止區明峰街往臺北方向行駛至該處，致原告人車倒地，原告2人因系爭事故受有系爭傷害等情，為被告所不爭執，原告此部分主張應堪信為真實。㈡按汽車臨時停車或停車，汽車駕駛人或乘客開啟或關閉車門時，應注意行人、其他車輛，並讓其先行。道路交通安全規則第112條第5項第3款定有明文。被告開啟車門未注意道路上行駛車輛，致原告甲○○騎乘之系爭機車撞擊車門後人車倒地，被告之行為顯有過失。次按因故意或過失，不法侵害他人之權利者，負損害賠償責任；汽車、機車或其他非依軌道行駛之動力車輛，在使用中加損害於他人者，駕駛人應賠償因此所生之損害。但於防止損害之發生，已盡相當之注意者，不在此限；又不法侵害他人之身體或健康者，對於被害人因此喪失或減少勞動能力或增加生活上之需要時，應負損害賠償責任；不法侵害他人身體、健康、名譽、自由、信用、隱私、貞操或不法侵害其他人格法益而情節重大者，被害人雖非財產上之損害，亦得請求賠償相當之金額；不法毀損他人之物者，被害人得請求賠償其物因毀損所減少之價額。民法第184條第1項前段、第191條之2、第193條第1項、第195條第1項前段、第196條分別定有明文。被告因前開過失駕駛行為，致原告受有系爭傷害及財產損害，依上開法條規定，被告自應負侵權行為賠償責任。㈢關於原告請求損害賠償之金額，茲審酌如下：1.醫療費：⑴原告甲女主張於108年1月21、22日住院，住院費用及其後回診費用總計支出3,648元，業據其提出三軍總醫院醫療費用收據為證，惟被告否認此部分醫療費用支出與系爭車禍事故相關。依三軍總醫院108年2月1日診斷證明書所載（本院卷第44頁），原告甲女於108年1月21、22日至三軍總醫院住院，出院後分別於108年1月25日、同年2月1日回診追蹤。診斷證明書病名欄記載：「中樞性尿崩症併高血鈉症。腦部病灶原因待查，疑似中樞神經免疫系統發炎相關（？）。注意力不集中併影響學習。嘔吐原因待查；疑似頭部外傷。」另醫師囑言欄記載：「住院期間接受腦部核磁共振攝影檢查及腦電波檢查；因疑似中樞神經免疫系統發炎相關（？）需安排anti-AQP4Abtest。建議兒童神經及內分泌科定期追蹤，並注意尿崩等相關神經症狀。」就原告甲女係因疑似中樞神經免疫系統疾病至三軍總醫院就診治療。再依三軍總醫院109年9月9日院三醫勤字第1090101697號函稱：「林員（即原告甲女）疑似中樞免疫系統發炎之相關診斷與車禍無關。」（見本院卷第164頁），及該院109年10月30日院三醫勤字第1090111903號函稱：「林員因自身未明原因之疑似脫髓鞘神經病變相關腦炎，可能復發導致反覆嘔吐、頭暈及尿崩症等情形，經評估確實需休養一個月。因上述症狀，該員類似臨床症狀安排住院，其主要住院原因為疑似中樞神經免疫系統發炎之復發；林員後續亦因疑似中樞神經免疫系統發炎之可能復發回診持續追蹤。」（見本院卷第184頁）。依三軍總醫院上開函覆內容，原告甲女自108年1月21日以後因疑似中樞神經免疫系統發炎之疾病住院與持續回診追蹤，且該疑似中樞神經免疫系統發炎與系爭車禍事故無關，是原告甲女請求此部分之醫療費用，並無依據。⑵原告甲○○主張支出醫療費用7,234元，此部分為被告所不爭執（見本院卷第37頁筆錄），是原告請求此部分之醫療費用，應予准許。⒉看護費用：⑴原告甲女主張因系爭傷害需休養一個月，而由父親照顧，故請求看護費用6萬元云云，但依三軍總醫院上開三軍總醫院函文，原告甲女雖因疑似中樞神經免疫系統發炎而有休養一個月之必要，惟上開疾病既與系爭交通事故無關，原告甲女此部分請求並無理由。⑵原告甲○○主張因系爭傷害需休養5天，因由配偶照顧，故請求看護費用1萬元等語。按親屬代為照顧被害人之起居，固係基於親情，但親屬看護所付出之勞力並非不能評價為金錢，雖因二者身分關係而免除被害人之支付義務，惟此種基於身分關係之恩惠，自不能加惠於加害人。故由親屬看護時雖無現實看護費之支付，仍應認被害人受有相當於看護費之損害，得向加害人請求賠償，始符公平原則（最高法院94年度台上字第1543號判決意旨參照）。依三軍總醫院109年5月19日診斷證明書（本願卷第46頁）醫師囑言欄記載：「民國107年12月21日病患於門診就診時診斷證明開立宜專人照顧一個月及休養兩個月」，故原告甲○○主張因系爭傷害由其配偶照顧5日，應可採信。又兩造不爭執看護費用以每日2,000元（見本院卷第37頁筆錄），是原告甲○○請求看護費用1萬元，應予准許。3.營養補充品費用：原告主張於受傷後，為加速復原，購買補充鈣質、甲殼素等營養品，總計支出37,350元云云，固提出統一發票、交易明細數紙為證【108年度審交附民字第426號卷（下稱附民卷）第43、45頁】。依原告所提收據中，僅有108年1月7日交易明細記載購買項目「健康食妍離子植物鈣」外，其餘均未記載購買項目，難認原告確有購買其所稱之營養品。況就原告所指購買之營養品，未有醫師醫囑指定為傷勢復原所必要，故亦難認為屬必要費用之支出，是原告甲○○此部分請求，為無理由。⒋原告甲○○主張因系爭交通事故致其機車受有損害，因修復機車總計支出1萬4,260元，業據其提出育軍機車行統一發票、估價單為證（本院卷第52、212頁），且為被告所不爭執（見本院卷第238頁筆錄）。另按不法毀損他人之物者，被害人得請求賠償其物因毀損所減少之價額，民法第196條定有明文，而所謂請求賠償物被毀損所減少之價額，得以修復費用為估定之標準，但以必要者為限，例如修理材料以新品換舊品，應予折舊。惟原告機車修復所使用之零件，並未標明係以中古品為之，常情應係以新零件更換損壞之舊零件，揆諸前揭說明，自應將零件折舊部分予以扣除。經查，原告之機車於103年4月出廠（日期未確定則以15日為準）距離系爭交通事故發生時間即107年10月23日，使用期間為4年6月9日，依行政院所頒「固定資產耐用年數表」及「固定資產折舊率表」之規定，機車之耐用年數為3年，依定率遞減法每年折舊千分之536；依98年9月14日修正前營利事業所得稅查核準則第95條第6項、第8項規定，固定資產提列折舊採用定率遞減法則者，以一年為計算單位，其使用期限未滿一年者，按實際使用之月數相當於全年之比例計算之，不滿一月者，以月計。又其最後一年之折舊額，加歷年折舊累積額，其總和不得超過該資產成本原額之10分之9。原告機車實際使用時間已超過3年之耐用年數，除其中車手校正（650元）、三角台校正（1,100元），及車台、右踏側及左邊板金校正（2,900元），總計4,650元應屬工資費用而不扣除折舊外，其餘零件費用9,610元（00000-0000）之折舊額應為8,649元（9610×0.9＝8649），扣除零件折舊額後，機車修復費用損失應為5,611元（00000-0000）。⒌工作損失：原告主張因系爭傷害向公司請假多日，遭扣薪3萬9,840元，有精華光學股份有限公司109年9月7日109精（管）外字第109090702號函在卷可稽（本院卷第156至162頁），且為被告所不爭執（見本院卷第237頁筆錄），是原告甲○○此部分請求應予准許。⒍勞動能力減損：原告甲○○主張因系爭交通事故導致勞動能力減損，受有77萬1,633元之損失等語。就原告甲○○勞動能力減損程度之評估，經囑託臺北榮民總醫院鑑定，依該院110年1月21日北總復字第1102000007號函檢送之工作功能評估報告（本院卷第186至187之3頁），認「個案倪○宜小姐於評估過程中，可理解、回應與依循評估治療師的要求與指令作業。本評估乃依據功能性能力評量法評估結果顯示，就個案隱形眼鏡產線作業員之工作而言，因受傷後手指主動角度受限，影響手部操作之速度，所以個案的工作功能受限程度約1%～10%。」以民法第193條第1項所謂減少勞動能力，乃指職業上工作能力一部之滅失而言。故審核被害人減少勞動能力之程度時，自應斟酌被害人之職業、智能、年齡、身體或健康狀態等各種因素，進而判斷其減少及殘存勞動能力之價值。本院斟酌原告甲○○為右手中指主動關節角度受限，依其自陳為高職畢業，從事作業員工作，主要仰賴手部作業，但依臺北榮民總醫院工作功能評估報告所載，其工作功能受到一些限制，可自行調整使得工作表現與效能不受限等情，認原告甲○○之勞動能力減損比例為5%。原告甲○○係70年4月22日出生，依其106年度綜合所得稅各類所得資料清單（附民卷第49頁），原告甲○○之薪資總額為45萬9,184元，故其每年減損勞動能力損害為2萬2,959元（459184×5%≒22959），原告請求自107年10月23日起至依勞動基準法第54條規定滿65歲強制退休即135年4月22日止，依霍夫曼式計算法扣除中間利息（首期給付不扣除中間利息）核計其金額為40萬3,848元【計算方式為：22,959×17.00000000+(22,959×0.00000000)×(17.00000000-00.00000000)=403,848.0000000000。其中17.00000000為年別單利5%第27年霍夫曼累計係數，17.00000000為年別單利5%第28年霍夫曼累計係數，0.00000000為未滿一年部分折算年數之比例(181/365=0.00000000)】，是原告得請求減損勞動能力所受損害為40萬3,848元。⒎慰撫金：按慰藉金之多寡，應斟酌雙方之身分、地位、資力與加害之程度及其他各種情形核定相當之數額。其金額是否相當，自應依實際加害情形與被害人所受之痛苦及雙方之身分、地位、經濟狀況等關係決定之（最高法院85年度台上字第460號判決意旨參照）。原告甲女自陳現為國中二年級學生，名下無不動產及股票投資；原告甲○○自陳高職畢業，現擔任作業員，每月收入約2萬4千餘元，名下僅有汽車1輛，並無不動產及股票投資；被告自陳護校畢業，月薪約33,000元，名下僅有股票投資（見本院卷第210頁民事綜合辯論意旨狀、第238頁筆錄），並有兩造之稅務電子閘門財產所得調件明細表在卷可參。本院斟酌上情，就原告2人因系爭事故所受傷勢程度，依其精神與肉體上之所受苦楚等一切情狀，認原告甲女、甲○○請求因系爭交通事故所受精神上之損害賠償各以4萬元、25萬元應屬相當，原告逾此部分之請求，則無理由。⒏綜上，原告甲女因系爭交通事故所受損害總計4萬元；原告甲○○因系爭交通事故所受損害總計71萬6,533元（7234+10000+5611+39840+403848+250000）㈣按保險人依本法規定所為之保險給付，視為被保險人損害賠償金額之一部分；被保險人受賠償請求時，得扣除之，強制汽車責任保險法第32條定有明文。經查，原告甲女、甲○○因系爭交通事故業已領取強制汽車責任保險金110元、14萬7,194元，被告並於刑案案件審理中先行賠償原告甲女、甲○○3萬元、7萬元（見本院卷第38、39頁筆錄），揆諸前開說明，此部分應自被告賠償金額中扣除。準此，原告甲女得請求賠償金額為9,890元（40000－30110），原告甲○○得請求賠償金額為49萬9,339元（000000-000000-00000）。㈤次按給付無確定期限者，債務人於債權人得請求給付時，經其催告而未為給付，自受催告時起，負遲延責任；其經債權人起訴而送達訴狀，或依督促程序送達支付命令，或為其他相類之行為者，與催告有同一之效力；遲延之債務，以支付金錢為標的者，債權人得請求依法定利率計算之遲延利息；應付利息之債務，其利率未經約定，亦無法律可據者，週年利率為百分之5。民法第229條第2項、第233條第1項本文及第203條分別定有明文。本件原告請求被告給付一定之金錢，屬以支付金錢為標的，依上開規定，原告請求被告給付自起訴狀繕本送達即108年10月8日（見附民卷第51頁送達證書）之翌日起，按週年利率百分之5計算之遲延利息，即無不合。四、綜上所述，原告2人基於侵權行為法律關係，請求被告給付原告甲女9,890元、原告甲○○49萬9,339元，及均自108年10月9日起至清償日止，按週年利率百分之5計算之利息，為有理由。至原告逾上開範圍之請求，則無理由，應予駁回。五、兩造均陳明願供擔保，請准宣告假執行及免為假執行。就原告勝訴部分，經核於法要無不合，爰分別酌定相當之擔保金額，予以准許。至原告敗訴部分，其假執行之聲請因失所附麗，應併予駁回之。六、本件事證已臻明確，兩造其餘之攻擊或防禦方法及未經援用之證據，經本院斟酌後，認為均不足以影響本判決之結果，自無一一詳予論駁之必要，併此敘明。七、據上論結，原告之訴為一部有理由、一部無理由，依民事訴訟法第436條第2項、第79條、第390條第2項、第390條第2項，判決如主文。中華民國110年4月7日民事第二庭法官辜漢忠以上正本係照原本作成。如對本判決上訴，應於判決送達後20日內向本院提出上訴狀，若委任律師提起上訴者，應一併繳納上訴審裁判費，否則本院得不命補正逕行駁回上訴。中華民國110年4月7日書記官潘盈筠"

In [12]:
test[3655]
import json
with open('../labelstudio/ml_backend_tmp_decanno.jsonl', encoding='utf-8') as f:
    s = json.load(f)
print(s)


JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)